# Home

In [ ]:
#!/usr/bin/env python

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
cd /content/drive/MyDrive/ColabNotebooks/Others/graph-2-text

/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text


In [ ]:
# print(torch.__version__)

In [ ]:
# !pip install torch==1.4.0 torchvision==0.5.0 torchtext==0.5.0

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from __future__ import division, unicode_literals
import os
import argparse
import math
import codecs
import torch

In [3]:
from itertools import count

In [4]:
import onmt.io
import onmt.translate
import onmt
import onmt.ModelConstructor
import onmt.modules
import opts

In [5]:
parser = argparse.ArgumentParser(
    description='translate.py',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
opts.add_md_help_argument(parser)
opts.translate_opts(parser)

In [10]:
opt = parser.parse_args("-model data/tmp__acc_69.51_ppl_3.43_e9.pt -data_type gcn -src data/webnlg/dev-webnlg-all-delex-src-nodes.txt -tgt data/webnlg/dev-webnlg-all-delex-tgt.txt -src_label data/webnlg/dev-webnlg-all-delex-src-labels.txt -src_node1 data/webnlg/dev-webnlg-all-delex-src-node1.txt -src_node2 data/webnlg/dev-webnlg-all-delex-src-node2.txt -output data/webnlg/delexicalized_predictions_dev.txt -replace_unk -verbose".split())

In [7]:
opt = parser.parse_args("-gpu 0 -model data/tmp__acc_69.51_ppl_3.43_e9.pt -data_type gcn -src data/webnlg/dev-webnlg-all-delex-src-nodes.txt -tgt data/webnlg/dev-webnlg-all-delex-tgt.txt -src_label data/webnlg/dev-webnlg-all-delex-src-labels.txt -src_node1 data/webnlg/dev-webnlg-all-delex-src-node1.txt -src_node2 data/webnlg/dev-webnlg-all-delex-src-node2.txt -output data/webnlg/delexicalized_predictions_dev.txt -replace_unk -verbose".split())

In [11]:
def _report_score(name, score_total, words_total):
    print("%s AVG SCORE: %.4f, %s PPL: %.4f" % (
        name, score_total / words_total,
        name, math.exp(-score_total / words_total)))

In [12]:
def _report_bleu():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    print()
    res = subprocess.check_output(
        "perl %s/tools/multi-bleu.perl %s < %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(">> " + res.strip())

In [13]:
def _report_rouge():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    res = subprocess.check_output(
        "python %s/tools/test_rouge.py -r %s -c %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(res.strip())

In [14]:
def main():
    dummy_parser = argparse.ArgumentParser(description='train.py')
    opts.model_opts(dummy_parser)
    dummy_opt = dummy_parser.parse_known_args([])[0]
    # opt.cuda = opt.gpu > -1
    opt.cuda = opt.gpu!=-1
    if opt.cuda:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # torch.cuda.set_device('cuda:0')
    else:
        device = torch.device("cpu")

    # Load the model.
    fields, model, model_opt = \
        onmt.ModelConstructor.load_test_model(opt, dummy_opt.__dict__)

    # File to write sentences to.
    out_file = codecs.open(opt.output, 'w', 'utf-8')

    # Test data
    if opt.data_type == 'gcn':
        data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                 opt.src, opt.tgt, opt.src_label,
                                 opt.src_node1, opt.src_node2,
                                 opt.src_morph,
                                 src_dir=opt.src_dir,
                                 sample_rate=opt.sample_rate,
                                 window_size=opt.window_size,
                                 window_stride=opt.window_stride,
                                 window=opt.window,
                                 use_filter_pred=False)
    else:
        data = onmt.io.build_dataset(fields, opt.data_type,
                                 opt.src, opt.tgt,
                                 src_dir=opt.src_dir,
                                 sample_rate=opt.sample_rate,
                                 window_size=opt.window_size,
                                 window_stride=opt.window_stride,
                                 window=opt.window,
                                 use_filter_pred=False)

    # Sort batch by decreasing lengths of sentence required by pytorch.
    # sort=False means "Use dataset's sortkey instead of iterator's".
    data_iter = onmt.io.OrderedIterator(
        dataset=data, device=device,
        batch_size=opt.batch_size, train=False, sort=False,
        sort_within_batch=True, shuffle=False)
    
    # Translator
    scorer = onmt.translate.GNMTGlobalScorer(opt.alpha,
                                             opt.beta,
                                             opt.coverage_penalty,
                                             opt.length_penalty)
    translator = onmt.translate.Translator(
        model, fields,
        beam_size=opt.beam_size,
        n_best=opt.n_best,
        global_scorer=scorer,
        max_length=opt.max_length,
        copy_attn=model_opt.copy_attn,
        cuda=opt.cuda,
        beam_trace=opt.dump_beam != "",
        min_length=opt.min_length,
        stepwise_penalty=opt.stepwise_penalty)
    builder = onmt.translate.TranslationBuilder(
        data, translator.fields,
        opt.n_best, opt.replace_unk, opt.tgt)

    # Statistics
    counter = count(1)
    pred_score_total, pred_words_total = 0, 0
    gold_score_total, gold_words_total = 0, 0
    for i, batch in enumerate(data_iter):
        print(i)
        print(batch.src)
        print(batch.node1)
        batch_data = translator.translate_batch(batch, data)
        translations = builder.from_batch(batch_data)
        for trans in translations:
            pred_score_total += trans.pred_scores[0]
            pred_words_total += len(trans.pred_sents[0])
            if opt.tgt:
                gold_score_total += trans.gold_score
                gold_words_total += len(trans.gold_sent) + 1
            n_best_preds = [" ".join(pred)
                            for pred in trans.pred_sents[:opt.n_best]]
            out_file.write('\n'.join(n_best_preds))
            out_file.write('\n')
            out_file.flush()
            if opt.verbose:
                sent_number = next(counter)
                output = trans.log(sent_number)
                os.write(1, output.encode('utf-8'))
    _report_score('PRED', pred_score_total, pred_words_total)
    if opt.tgt:
        _report_score('GOLD', gold_score_total, gold_words_total)
        if opt.report_bleu:
            _report_bleu()
        if opt.report_rouge:
            _report_rouge()
    if opt.dump_beam:
        import json
        json.dump(translator.beam_accum,
                  codecs.open(opt.dump_beam, 'w', 'utf-8'))

In [ ]:
if __name__ == "__main__":
    main()

# One Batch TEST

In [ ]:
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]
# opt.cuda = opt.gpu > -1
opt.cuda = opt.gpu!='cpu'
if opt.cuda:
    torch.cuda.set_device(opt.gpu)
    # torch.cuda.set_device('cuda:0')

# Load the model.
fields, model, model_opt = \
    onmt.ModelConstructor.load_test_model(opt, dummy_opt.__dict__)

# File to write sentences to.
out_file = codecs.open(opt.output, 'w', 'utf-8')

# Test data
if opt.data_type == 'gcn':
    data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                opt.src, opt.tgt, opt.src_label,
                                opt.src_node1, opt.src_node2,
                                opt.src_morph,
                                src_dir=opt.src_dir,
                                sample_rate=opt.sample_rate,
                                window_size=opt.window_size,
                                window_stride=opt.window_stride,
                                window=opt.window,
                                use_filter_pred=False)
else:
    data = onmt.io.build_dataset(fields, opt.data_type,
                                opt.src, opt.tgt,
                                src_dir=opt.src_dir,
                                sample_rate=opt.sample_rate,
                                window_size=opt.window_size,
                                window_stride=opt.window_stride,
                                window=opt.window,
                                use_filter_pred=False)

# Sort batch by decreasing lengths of sentence required by pytorch.
# sort=False means "Use dataset's sortkey instead of iterator's".
data_iter = onmt.io.OrderedIterator(
    dataset=data, device=opt.gpu,
    batch_size=opt.batch_size, train=False, sort=False,
    sort_within_batch=True, shuffle=False)

# Translator
scorer = onmt.translate.GNMTGlobalScorer(opt.alpha,
                                            opt.beta,
                                            opt.coverage_penalty,
                                            opt.length_penalty)
translator = onmt.translate.Translator(
    model, fields,
    beam_size=opt.beam_size,
    n_best=opt.n_best,
    global_scorer=scorer,
    max_length=opt.max_length,
    copy_attn=model_opt.copy_attn,
    cuda=opt.cuda,
    beam_trace=opt.dump_beam != "",
    min_length=opt.min_length,
    stepwise_penalty=opt.stepwise_penalty)
builder = onmt.translate.TranslationBuilder(
    data, translator.fields,
    opt.n_best, opt.replace_unk, opt.tgt)

# Statistics
counter = count(1)
pred_score_total, pred_words_total = 0, 0
gold_score_total, gold_words_total = 0, 0
# for i, batch in enumerate(data_iter):
i=0
batch=data_iter.__iter__().__next__()
# print(i)
# print(batch.src)
# print(batch.node1)
batch_data = translator.translate_batch(batch, data)
translations = builder.from_batch(batch_data)
for trans in translations:
    print("trans",str(trans))
    print("trans.pred_scores[0]",str(trans.pred_scores))
    pred_score_total += trans.pred_scores[0]
    pred_words_total += len(trans.pred_sents[0])
    if opt.tgt:
        gold_score_total += trans.gold_score
        gold_words_total += len(trans.gold_sent) + 1
    n_best_preds = [" ".join(pred)
                    for pred in trans.pred_sents[:opt.n_best]]
    print("n_best_preds",str(n_best_preds))
    out_file.write('\n'.join(n_best_preds))
    out_file.write('\n')
    out_file.flush()
    if opt.verbose:
        sent_number = next(counter)
        output = trans.log(sent_number)
        os.write(1, output.encode('utf-8'))
_report_score('PRED', pred_score_total, pred_words_total)
if opt.tgt:
    _report_score('GOLD', gold_score_total, gold_words_total)
    if opt.report_bleu:
        _report_bleu()
    if opt.report_rouge:
        _report_rouge()
if opt.dump_beam:
    import json
    json.dump(translator.beam_accum,
                codecs.open(opt.dump_beam, 'w', 'utf-8'))

use gates =  False


/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:230: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_1)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:232: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_2)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:234: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_3)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:236: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_4)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/my_modules/GCN.py:41: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_norma

Loading model parameters.
average src size 8.116402116402117 2268


/usr/local/lib/python3.7/dist-packages/torchtext/legacy/data/field.py:353: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/translate/Translator.py:87: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  def var(a): return Variable(a, volatile=True)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:901: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  for e in self._all]
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(a

trans <onmt.translate.Translation.Translation object at 0x7f3d434d8910>
trans.pred_scores[0] [tensor(-10.1041)]
n_best_preds ['<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>']
PRED SCORE: -10.1041
GOLD SCORE: -51.0763
trans <onmt.translate.Translation.Translation object at 0x7f3d434d84d0>
trans.pred_scores[0] [tensor(-10.1041)]
n_best_preds ['<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 

NameError: ignored

In [ ]:
translations[0].src

In [ ]:
translations[0].src_raw

('Aarhus', 'leaderName', 'LEADERNAME')

In [ ]:
model_opt.copy_attn

False

In [ ]:
fields["tgt"].vocab.stoi["<unk>"]

0

In [ ]:
translations[0].pred_sents

In [ ]:
translations[0].attns

In [ ]:
translations[0].gold_sent

['<unk>', 'leader', 'of', '<unk>', 'is', '<unk>', '.']

In [ ]:
translations[0].gold_score

tensor(-51.0763, device='cuda:0')

In [ ]:
translations[0].pred_sents[:opt.n_best]

# Translator.py TEST

In [ ]:
from torch.autograd import Variable
def translate_batch(self, batch, data):
    """
    Translate a batch of sentences.

    Mostly a wrapper around :obj:`Beam`.

    Args:
        batch (:obj:`Batch`): a batch from a dataset object
        data (:obj:`Dataset`): the dataset object


    Todo:
        Shouldn't need the original dataset.
    """

    # (0) Prep each of the components of the search.
    # And helper method for reducing verbosity.
    beam_size = self.beam_size
    batch_size = batch.batch_size
    data_type = data.data_type
    vocab = self.fields["tgt"].vocab
    beam = [onmt.translate.Beam(beam_size, n_best=self.n_best,
                                cuda=self.cuda,
                                global_scorer=self.global_scorer,
                                pad=vocab.stoi[onmt.io.PAD_WORD],
                                eos=vocab.stoi[onmt.io.EOS_WORD],
                                bos=vocab.stoi[onmt.io.BOS_WORD],
                                min_length=self.min_length,
                                stepwise_penalty=self.stepwise_penalty)
            for __ in range(batch_size)]
    
    # Help functions for working with beams and batches
    def var(a): return Variable(a, volatile=True)

    def rvar(a): return var(a.repeat(1, beam_size, 1))

    def bottle(m):
        return m.view(batch_size * beam_size, -1)

    def unbottle(m):
        return m.view(beam_size, batch_size, -1)

    # (1) Run the encoder on the src.
    src = onmt.io.make_features(batch, 'src', data_type)
    src_lengths = None
    if data_type == 'text':
        _, src_lengths = batch.src
    elif data_type == 'gcn':
        _, src_lengths = batch.src
        # report_stats.n_src_words += src_lengths.sum()
        adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
        mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
        if hasattr(batch, 'morph'):
            morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

    if data_type == 'gcn':
        # F-prop through the model.
        if hasattr(batch, 'morph'):
            enc_states, memory_bank = \
                self.model.encoder(src, src_lengths,
                                    adj_arc_in, adj_arc_out, adj_lab_in,
                                    adj_lab_out, mask_in, mask_out,
                                    mask_loop, mask_sent, morph, mask_morph)
        else:
            enc_states, memory_bank = \
                self.model.encoder(src, src_lengths,
                            adj_arc_in, adj_arc_out, adj_lab_in,
                            adj_lab_out, mask_in, mask_out,
                            mask_loop, mask_sent)
    else:
        enc_states, memory_bank = self.model.encoder(src, src_lengths)

    # print("memory_bank:",str(memory_bank))
    # print("memory_bank.data:",str(memory_bank.data))
    # print("size memory_bank:",str(memory_bank.size()))
    # print("size memory_bank.data:",str(memory_bank.size()))

    dec_states = self.model.decoder.init_decoder_state(
                                    src, memory_bank, enc_states)

    if src_lengths is None:
        src_lengths = torch.Tensor(batch_size).type_as(memory_bank.data)\
                                                .long()\
                                                .fill_(memory_bank.size(0))

    # (2) Repeat src objects `beam_size` times.
    src_map = rvar(batch.src_map.data) \
        if (data_type == 'text' or data_type == 'gcn') and self.copy_attn else None
    memory_bank = rvar(memory_bank.data)
    memory_lengths = src_lengths.repeat(beam_size)
    dec_states.repeat_beam_size_times(beam_size)

    # (3) run the decoder to generate sentences, using beam search.
    for i in range(self.max_length):
        if all((b.done() for b in beam)):
            break

        # Construct batch x beam_size nxt words.
        # Get all the pending current beam words and arrange for forward.
        # print("beam:",beam)
        # print("len beam:", str(len(beam)))
        inp = var(torch.stack([b.get_current_state() for b in beam])
                    .t().contiguous().view(1, -1))
        # inp = rvar(inp)
        # print("size inp:",str(inp.size()))
        # inp = inp.repeat(1,beam_size)
        # print("size inp:",str(inp.size()))


        # Turn any copied words to UNKs
        # 0 is unk
        if self.copy_attn:
            inp = inp.masked_fill(
                inp.gt(len(self.fields["tgt"].vocab) - 1), 0)

        # Temporary kludge solution to handle changed dim expectation
        # in the decoder
        inp = inp.unsqueeze(2)

        # Run one step.
        # print("inp:",str(inp))
        # print("size inp:",str(inp.size()))
        # print("memorybank;",str(memory_bank))
        # print("size memorybank:",str(memory_bank.size()))
        dec_out, dec_states, attn = self.model.decoder(
            inp, memory_bank, dec_states, memory_lengths=memory_lengths)
        dec_out = dec_out.squeeze(0)
        # dec_out: beam x rnn_size

        # (b) Compute a vector of batch x beam word scores.
        if not self.copy_attn:
            out = self.model.generator.forward(dec_out).data
            out = unbottle(out)
            # beam x tgt_vocab
            beam_attn = unbottle(attn["std"])
        else:
            out = self.model.generator.forward(dec_out,
                                                attn["copy"].squeeze(0),
                                                src_map)
            # beam x (tgt_vocab + extra_vocab)
            out = data.collapse_copy_scores(
                unbottle(out.data),
                batch, self.fields["tgt"].vocab, data.src_vocabs)
            # beam x tgt_vocab
            out = out.log()
            beam_attn = unbottle(attn["copy"])
        # (c) Advance each beam.
        for j, b in enumerate(beam):
            b.advance(out[:, j],
                        beam_attn.data[:, j, :memory_lengths[j]])
            dec_states.beam_update(j, b.get_current_origin(), beam_size)

    # (4) Extract sentences from beam.
    ret = self._from_beam(beam)
    ret["gold_score"] = [0] * batch_size
    if "tgt" in batch.__dict__:
        ret["gold_score"] = self._run_target(batch, data)
    ret["batch"] = batch
    return ret

In [ ]:
i=0
batch=data_iter.__iter__().__next__()
data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                opt.src, opt.tgt, opt.src_label,
                                opt.src_node1, opt.src_node2,
                                opt.src_morph,
                                src_dir=opt.src_dir,
                                sample_rate=opt.sample_rate,
                                window_size=opt.window_size,
                                window_stride=opt.window_stride,
                                window=opt.window,
                                use_filter_pred=False)

/usr/local/lib/python3.7/dist-packages/torchtext/legacy/data/field.py:353: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


average src size 8.116402116402117 2268


In [ ]:
translate_batch(translator, batch, data)

## inp.masked_fill TEST

In [ ]:
batch_size = batch.batch_size
beam_size = opt.beam_size
vocab = translator.fields["tgt"].vocab
beam = [onmt.translate.Beam(beam_size, n_best=translator.n_best,
                                cuda=translator.cuda,
                                global_scorer=translator.global_scorer,
                                pad=vocab.stoi[onmt.io.PAD_WORD],
                                eos=vocab.stoi[onmt.io.EOS_WORD],
                                bos=vocab.stoi[onmt.io.BOS_WORD],
                                min_length=translator.min_length,
                                stepwise_penalty=translator.stepwise_penalty)
            for __ in range(batch_size)]

In [ ]:
def var(a): return Variable(a, volatile=True)

def rvar(a): return var(a.repeat(1, beam_size, 1))

def bottle(m):
    return m.view(batch_size * beam_size, -1)

def unbottle(m):
    return m.view(beam_size, batch_size, -1)

In [ ]:
inp = var(torch.stack([b.get_current_state() for b in beam])
                    .t().contiguous().view(1, -1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """Entry point for launching an IPython kernel.


In [ ]:
print("inp",str(inp))
print("type inp", str(type(inp)))
print("size inp",str(inp.size()))

inp tensor([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]], device='cuda:0')
type inp <class 'torch.Tensor'>
size inp torch.Size([1, 150])


In [ ]:
inp = inp.masked_fill(
                inp.gt(len(translator.fields["tgt"].vocab) - 1), 0)

In [ ]:
print("inp",str(inp))
print("type inp", str(type(inp)))
print("size inp",str(inp.size()))

inp tensor([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]], device='cuda:0')
type inp <class 'torch.Tensor'>
size inp torch.Size([1, 150])


In [ ]:
inp = inp.unsqueeze(2)

In [ ]:
print("inp",str(inp))

##  translate_batch TEST in Translator.py

In [ ]:
"""
Translate a batch of sentences.

Mostly a wrapper around :obj:`Beam`.

Args:
    batch (:obj:`Batch`): a batch from a dataset object
    data (:obj:`Dataset`): the dataset object


Todo:
    Shouldn't need the original dataset.
"""

# (0) Prep each of the components of the search.
# And helper method for reducing verbosity.
beam_size = translator.beam_size
batch_size = batch.batch_size
data_type = data.data_type
vocab = translator.fields["tgt"].vocab
beam = [onmt.translate.Beam(beam_size, n_best=translator.n_best,
                            cuda=translator.cuda,
                            global_scorer=translator.global_scorer,
                            pad=vocab.stoi[onmt.io.PAD_WORD],
                            eos=vocab.stoi[onmt.io.EOS_WORD],
                            bos=vocab.stoi[onmt.io.BOS_WORD],
                            min_length=translator.min_length,
                            stepwise_penalty=translator.stepwise_penalty)
        for __ in range(batch_size)]

# Help functions for working with beams and batches
def var(a): return Variable(a, volatile=True)

def rvar(a): return var(a.repeat(1, beam_size, 1))

def bottle(m):
    return m.view(batch_size * beam_size, -1)

def unbottle(m):
    return m.view(beam_size, batch_size, -1)

# (1) Run the encoder on the src.
src = onmt.io.make_features(batch, 'src', data_type)
src_lengths = None
if data_type == 'text':
    _, src_lengths = batch.src
elif data_type == 'gcn':
    _, src_lengths = batch.src
    # report_stats.n_src_words += src_lengths.sum()
    adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
    mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
    if hasattr(batch, 'morph'):
        morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

if data_type == 'gcn':
    # F-prop through the model.
    if hasattr(batch, 'morph'):
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                                adj_arc_in, adj_arc_out, adj_lab_in,
                                adj_lab_out, mask_in, mask_out,
                                mask_loop, mask_sent, morph, mask_morph)
    else:
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                        adj_arc_in, adj_arc_out, adj_lab_in,
                        adj_lab_out, mask_in, mask_out,
                        mask_loop, mask_sent)
else:
    enc_states, memory_bank = translator.model.encoder(src, src_lengths)

# print("memory_bank:",str(memory_bank))
# print("memory_bank.data:",str(memory_bank.data))
# print("size memory_bank:",str(memory_bank.size()))
# print("size memory_bank.data:",str(memory_bank.size()))

dec_states = translator.model.decoder.init_decoder_state(
                                src, memory_bank, enc_states)

if src_lengths is None:
    src_lengths = torch.Tensor(batch_size).type_as(memory_bank.data)\
                                            .long()\
                                            .fill_(memory_bank.size(0))

# (2) Repeat src objects `beam_size` times.
src_map = rvar(batch.src_map.data) \
    if (data_type == 'text' or data_type == 'gcn') and translator.copy_attn else None
memory_bank = rvar(memory_bank.data)
memory_lengths = src_lengths.repeat(beam_size)
dec_states.repeat_beam_size_times(beam_size)

# (3) run the decoder to generate sentences, using beam search.
for i in range(translator.max_length):
    if all((b.done() for b in beam)):
        break

    # Construct batch x beam_size nxt words.
    # Get all the pending current beam words and arrange for forward.
    # print("beam:",beam)
    # print("len beam:", str(len(beam)))
    inp = var(torch.stack([b.get_current_state() for b in beam])
                .t().contiguous().view(1, -1))
    for b in beam:
        # print("b.currentState:",str(b.get_current_state()), end=' ')
    # inp = rvar(inp)
    # print("size inp:",str(inp.size()))
    # inp = inp.repeat(1,beam_size)
    # print("size inp:",str(inp.size()))


    # Turn any copied words to UNKs
    # 0 is unk
    if translator.copy_attn:
        inp = inp.masked_fill(
            inp.gt(len(translator.fields["tgt"].vocab) - 1), 0)

    # Temporary kludge solution to handle changed dim expectation
    # in the decoder
    inp = inp.unsqueeze(2)

    # Run one step.
    # print("inp:",str(inp))
    # print("size inp:",str(inp.size()))
    # print("memorybank;",str(memory_bank))
    # print("size memorybank:",str(memory_bank.size()))
    dec_out, dec_states, attn = translator.model.decoder(
        inp, memory_bank, dec_states, memory_lengths=memory_lengths)
    dec_out = dec_out.squeeze(0)
    # dec_out: beam x rnn_size

    # (b) Compute a vector of batch x beam word scores.
    if not translator.copy_attn:
        out = translator.model.generator.forward(dec_out).data
        out = unbottle(out)
        # beam x tgt_vocab
        beam_attn = unbottle(attn["std"])
    else:
        out = translator.model.generator.forward(dec_out,
                                            attn["copy"].squeeze(0),
                                            src_map)
        # beam x (tgt_vocab + extra_vocab)
        out = data.collapse_copy_scores(
            unbottle(out.data),
            batch, translator.fields["tgt"].vocab, data.src_vocabs)
        # beam x tgt_vocab
        out = out.log()
        beam_attn = unbottle(attn["copy"])
    # (c) Advance each beam.
    for j, b in enumerate(beam):
        b.advance(out[:, j],
                    beam_attn.data[:, j, :memory_lengths[j]])
        dec_states.beam_update(j, b.get_current_origin(), beam_size)

# (4) Extract sentences from beam.
ret = translator._from_beam(beam)
ret["gold_score"] = [0] * batch_size
if "tgt" in batch.__dict__:
    ret["gold_score"] = translator._run_target(batch, data)
ret["batch"] = batch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:901: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  for e in self._all]
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1, 1, 1], device='cuda:0') b.currentState: tensor([2, 1, 1

In [ ]:
print(ret['predictions'])

[[[tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., 

# Beam.py TEST

In [20]:
from torch.autograd import Variable

In [21]:
from __future__ import division
import torch
from onmt.translate import Penalties


class Beam(object):
    """
    Class for managing the internals of the beam search process.

    Takes care of beams, back pointers, and scores.

    Args:
        size (int): beam size
        pad, bos, eos (int): indices of padding, beginning, and ending.
        n_best (int): nbest size to use
        cuda (bool): use gpu
        global_scorer (:obj:`GlobalScorer`)
    """
    def __init__(self, size, pad, bos, eos,
                    n_best=1, cuda=False,
                    global_scorer=None,
                    min_length=0,
                    stepwise_penalty=False):

        self.size = size
        # self.tt = torch.cuda if cuda else torch
        self.tt = 'cuda:0' if cuda else 'cpu'

        # The score for each translation on the beam.
        # self.scores = self.tt.FloatTensor(size).zero_()
        # self.scores = self.tt.tensor(size,dtype=torch.float).zero_()
        self.scores = torch.tensor(size,dtype=torch.float,device=self.tt).zero_()
        self.all_scores = []

        # The backpointers at each time-step.
        self.prev_ks = []

        # The outputs at each time-step.
        # self.next_ys = [self.tt.LongTensor(size)
        #                 .fill_(pad)]
        # self.next_ys = [self.tt.tensor((1,size),dtype=torch.long)
        #                 .fill_(pad)]
        # self.next_ys = [self.tt.tensor([]).new_full((5,),pad,dtype=torch.long)]
        self.next_ys = [torch.tensor([]).new_full((5,),pad,dtype=torch.long, device=self.tt)]
        self.next_ys[0][0] = bos
        # self.next_ys = bos

        # Has EOS topped the beam yet.
        self._eos = eos
        self.eos_top = False

        # The attentions (matrix) for each time.
        self.attn = []

        # Time and k pair for finished.
        self.finished = []
        self.n_best = n_best

        # Information for global scoring.
        self.global_scorer = global_scorer
        self.global_state = {}

        # Minimum prediction length
        self.min_length = min_length

        # Apply Penalty at every step
        self.stepwise_penalty = stepwise_penalty

    def get_current_state(self):
        "Get the outputs for the current timestep."
        return self.next_ys[-1]

    def get_current_origin(self):
        "Get the backpointers for the current timestep."
        return self.prev_ks[-1]

    def advance(self, word_probs, attn_out):
        """
        Given prob over words for every last beam `wordLk` and attention
        `attn_out`: Compute and update the beam search.

        Parameters:

        * `word_probs`- probs of advancing from the last step (K x words)
        * `attn_out`- attention at the last step

        Returns: True if beam search is complete.
        """
        num_words = word_probs.size(1)
        if self.stepwise_penalty:
            self.global_scorer.update_score(self, attn_out)
        # force the output to be longer than self.min_length
        cur_len = len(self.next_ys)
        if cur_len < self.min_length:
            for k in range(len(word_probs)):
                word_probs[k][self._eos] = -1e20
        # Sum the previous scores.
        if len(self.prev_ks) > 0:
            beam_scores = word_probs + \
                self.scores.unsqueeze(1).expand_as(word_probs)
            # Don't let EOS have children.
            for i in range(self.next_ys[-1].size(0)):
                if self.next_ys[-1][i] == self._eos:
                    beam_scores[i] = -1e20
        else:
            beam_scores = word_probs[0]
        flat_beam_scores = beam_scores.view(-1)
        best_scores, best_scores_id = flat_beam_scores.topk(self.size, 0,
                                                            True, True)

        self.all_scores.append(self.scores)
        self.scores = best_scores

        # best_scores_id is flattened beam x word array, so calculate which
        # word and beam each score came from
        prev_k = best_scores_id / num_words
        self.prev_ks.append(prev_k)
        self.next_ys.append((best_scores_id - prev_k * num_words))
        # print("next_ys[-1]",str(self.next_ys[-1]))
        self.attn.append(attn_out)
        self.global_scorer.update_global_state(self)

        for i in range(self.next_ys[-1].size(0)):
            if self.next_ys[-1][i] == self._eos:
                global_scores = self.global_scorer.score(self, self.scores)
                s = global_scores[i]
                self.finished.append((s, len(self.next_ys) - 1, i))

        # End condition is when top-of-beam is EOS and no global score.
        if self.next_ys[-1][0] == self._eos:
            self.all_scores.append(self.scores)
            self.eos_top = True

    def done(self):
        return self.eos_top and len(self.finished) >= self.n_best

    def sort_finished(self, minimum=None):
        if minimum is not None:
            i = 0
            # Add from beam until we have minimum outputs.
            while len(self.finished) < minimum:
                global_scores = self.global_scorer.score(self, self.scores)
                s = global_scores[i]
                self.finished.append((s, len(self.next_ys) - 1, i))
                i += 1

        self.finished.sort(key=lambda a: -a[0])
        scores = [sc for sc, _, _ in self.finished]
        ks = [(t, k) for _, t, k in self.finished]
        return scores, ks

    def get_hyp(self, timestep, k):
        """
        Walk back to construct the full hypothesis.
        """
        hyp, attn = [], []
        for j in range(len(self.prev_ks[:timestep]) - 1, -1, -1):
            # print("type(self.next_ys[j+1][k])",str(type(self.next_ys[j+1][k])))
            # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].dtype))
            # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].type(torch.long).dtype))
            hyp.append(self.next_ys[j+1][int(k)])
            attn.append(self.attn[j][int(k)])
            k = self.prev_ks[j][int(k)]
        return hyp[::-1], torch.stack(attn[::-1])


class GNMTGlobalScorer(object):
    """
    NMT re-ranking score from
    "Google's Neural Machine Translation System" :cite:`wu2016google`

    Args:
        alpha (float): length parameter
        beta (float):  coverage parameter
    """
    def __init__(self, alpha, beta, cov_penalty, length_penalty):
        self.alpha = alpha
        self.beta = beta
        penalty_builder = Penalties.PenaltyBuilder(cov_penalty,
                                                    length_penalty)
        # Term will be subtracted from probability
        self.cov_penalty = penalty_builder.coverage_penalty()
        # Probability will be divided by this
        self.length_penalty = penalty_builder.length_penalty()

    def score(self, beam, logprobs):
        """
        Rescores a prediction based on penalty functions
        """
        normalized_probs = self.length_penalty(beam,
                                                logprobs,
                                                self.alpha)
        if not beam.stepwise_penalty:
            penalty = self.cov_penalty(beam,
                                        beam.global_state["coverage"],
                                        self.beta)
            normalized_probs -= penalty

        return normalized_probs

    def update_score(self, beam, attn):
        """
        Function to update scores of a Beam that is not finished
        """
        if "prev_penalty" in beam.global_state.keys():
            beam.scores.add_(beam.global_state["prev_penalty"])
            penalty = self.cov_penalty(beam,
                                        beam.global_state["coverage"] + attn,
                                        self.beta)
            beam.scores.sub_(penalty)

    def update_global_state(self, beam):
        "Keeps the coverage vector as sum of attentions"
        if len(beam.prev_ks) == 1:
            beam.global_state["prev_penalty"] = beam.scores.clone().fill_(0.0)
            beam.global_state["coverage"] = beam.attn[-1]
            self.cov_total = beam.attn[-1].sum(1)
        else:
            self.cov_total += torch.min(beam.attn[-1],
                                        beam.global_state['coverage']).sum(1)
            # print("beam.globa._state",str(beam.global_state["coverage"]))
            # print("size beam.global_state",str(beam.global_state["coverage"].size()))
            # print("size beam.prev_ks[-1]",str(beam.prev_ks[-1].size()))
            beam.global_state["coverage"] = beam.global_state["coverage"] \
                .index_select(0, (beam.prev_ks[-1]/4623).type(torch.int64)).add(beam.attn[-1])

            prev_penalty = self.cov_penalty(beam,
                                            beam.global_state["coverage"],
                                            self.beta)
            beam.global_state["prev_penalty"] = prev_penalty


## def　advance UI test

In [ ]:
batch=iter(data_iter).__next__()



/usr/local/lib/python3.7/dist-packages/torchtext/legacy/data/field.py:353: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


In [ ]:
i=0
batch=iter(data_iter).__next__()
beam_size = translator.beam_size
batch_size = batch.batch_size
data_type = data.data_type
vocab = translator.fields["tgt"].vocab
beam = [Beam(beam_size, n_best=translator.n_best,
                            cuda=translator.cuda,
                            global_scorer=translator.global_scorer,
                            pad=vocab.stoi[onmt.io.PAD_WORD],
                            eos=vocab.stoi[onmt.io.EOS_WORD],
                            bos=vocab.stoi[onmt.io.BOS_WORD],
                            min_length=translator.min_length,
                            stepwise_penalty=translator.stepwise_penalty)
        for __ in range(batch_size)]

/usr/local/lib/python3.7/dist-packages/torchtext/legacy/data/field.py:353: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


NameError: ignored

In [ ]:
for j, b in enumerate(beam):
    b.advance(out[:, j],
                beam_attn.data[:, j, :memory_lengths[j]])
    dec_states.beam_update(j, b.get_current_origin(), beam_size)

## def advance IT TEST

In [22]:
def advance(self, word_probs, attn_out):
        """
        Given prob over words for every last beam `wordLk` and attention
        `attn_out`: Compute and update the beam search.

        Parameters:

        * `word_probs`- probs of advancing from the last step (K x words)
                     -- K is beam size
                     -- words is tgt vocabulary size

        * `attn_out`- attention at the last step
                    - size 5 * 3

        Returns: True if beam search is complete.
        """
        # print("attn_out",str(attn_out))
        # print("size attn_out",str(attn_out.size()))
        num_words = word_probs.size(1)
        if self.stepwise_penalty:
            self.global_scorer.update_score(self, attn_out)
        # force the output to be longer than self.min_length
        cur_len = len(self.next_ys)
        if cur_len < self.min_length:
            for k in range(len(word_probs)):
                word_probs[k][self._eos] = -1e20
        # Sum the previous scores.
        if len(self.prev_ks) > 0:
            beam_scores = word_probs + \
                self.scores.unsqueeze(1).expand_as(word_probs)
            # Don't let EOS have children.
            for i in range(self.next_ys[-1].size(0)):
                if self.next_ys[-1][i] == self._eos:
                    beam_scores[i] = -1e20
        else:
            beam_scores = word_probs[0]
        # print("beam_scores",str(beam_scores))
        # print("size beam_scores",str(beam_scores.size()))
        flat_beam_scores = beam_scores.view(-1)
        best_scores, best_scores_id = flat_beam_scores.topk(self.size, 0,
                                                            True, True)

        self.all_scores.append(self.scores)
        self.scores = best_scores

        # best_scores_id is flattened beam x word array, so calculate which
        # word and beam each score came from
        # print("best_score",str(best_scores))
        # print("best_socre_id",str(best_scores_id))
        prev_k = (best_scores_id / num_words).type(torch.int64)
        # print("prev_k",str(prev_k))
        self.prev_ks.append(prev_k)
        # self.next_ys.append((best_scores_id - prev_k * num_words))
        self.next_ys.append(best_scores_id % num_words)
        self.attn.append(attn_out)
        self.global_scorer.update_global_state(self)

        for i in range(self.next_ys[-1].size(0)):
            if self.next_ys[-1][i] == self._eos:
                global_scores = self.global_scorer.score(self, self.scores)
                s = global_scores[i]
                self.finished.append((s, len(self.next_ys) - 1, i))

        # End condition is when top-of-beam is EOS and no global score.
        if self.next_ys[-1][0] == self._eos:
            self.all_scores.append(self.scores)
            self.eos_top = True

        # print("self.next_ys",str(self.next_ys))

## def from_beam TEST

In [23]:
def _from_beam(self, beam):
        ret = {"predictions": [],
               "scores": [],
               "attention": []}
        for b in beam:
            n_best = self.n_best
            scores, ks = sort_finished(b,minimum=n_best)
            hyps, attn = [], []
            for i, (times, k) in enumerate(ks[:n_best]):
                hyp, att = get_hyp(b,times, k)
                hyps.append(hyp)
                attn.append(att)
            ret["predictions"].append(hyps)
            ret["scores"].append(scores)
            ret["attention"].append(attn)
        return ret

## def sort_finished TEST

In [24]:
def sort_finished(self, minimum=None):
    if minimum is not None:
        i = 0
        # Add from beam until we have minimum outputs.
        while len(self.finished) < minimum:
            global_scores = self.global_scorer.score(self, self.scores)
            s = global_scores[i]
            self.finished.append((s, len(self.next_ys) - 1, i))
            i += 1

    self.finished.sort(key=lambda a: -a[0])
    scores = [sc for sc, _, _ in self.finished]
    ks = [(t, k) for _, t, k in self.finished]
    # print("self.finished",str(self.finished))
    return scores, ks

## def get_hyp TEST

In [25]:
def get_hyp(self, timestep, k):
    """
    Walk back to construct the full hypothesis.
    """
    hyp, attn = [], []
    # print("len(self.prev_ks[:timestep])",str(len(self.prev_ks[:timestep])))
    for j in range(len(self.prev_ks[:timestep]) - 1, -1, -1):
        # print("type(self.next_ys[j+1][k])",str(type(self.next_ys[j+1][k])))
        # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].dtype))
        # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].type(torch.long).dtype))
        # print("self.next_ys[j+1]",str(self.next_ys[j+1]))
        # print("self.next_ys[j+1]",str(self.next_ys[j+1]))
        # print("size self.next_ys[j+1]",str(self.next_ys[j+1].size()))
        # print("k",str(k))
        hyp.append(self.next_ys[j+1][int(k)])
        attn.append(self.attn[j][int(k)])
        k = self.prev_ks[j][int(k)]
        # print("k",k)
    # print("hyp",str(hyp))
    return hyp[::-1], torch.stack(attn[::-1])

## One Batch

In [26]:
from torch.autograd import Variable
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]
# opt.cuda = opt.gpu > -1
opt.cuda = opt.gpu!=-1
if opt.cuda:
    torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # torch.cuda.device(opt.gpu)
    # torch.cuda.set_device(opt.gpu)
    # torch.cuda.set_device('cuda:0')

# Load the model.
fields, model, model_opt = \
    onmt.ModelConstructor.load_test_model(opt, dummy_opt.__dict__)

# File to write sentences to.
out_file = codecs.open(opt.output, 'w', 'utf-8')

# Test data
if opt.data_type == 'gcn':
    data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                opt.src, opt.tgt, opt.src_label,
                                opt.src_node1, opt.src_node2,
                                opt.src_morph,
                                src_dir=opt.src_dir,
                                sample_rate=opt.sample_rate,
                                window_size=opt.window_size,
                                window_stride=opt.window_stride,
                                window=opt.window,
                                use_filter_pred=False)
else:
    data = onmt.io.build_dataset(fields, opt.data_type,
                                opt.src, opt.tgt,
                                src_dir=opt.src_dir,
                                sample_rate=opt.sample_rate,
                                window_size=opt.window_size,
                                window_stride=opt.window_stride,
                                window=opt.window,
                                use_filter_pred=False)

# Sort batch by decreasing lengths of sentence required by pytorch.
# sort=False means "Use dataset's sortkey instead of iterator's".
data_iter = onmt.io.OrderedIterator(
    dataset=data, device=opt.gpu,
    batch_size=opt.batch_size, train=False, sort=False,
    sort_within_batch=True, shuffle=False)

# Translator
scorer = GNMTGlobalScorer(opt.alpha,
                                            opt.beta,
                                            opt.coverage_penalty,
                                            opt.length_penalty)
translator = onmt.translate.Translator(
    model, fields,
    beam_size=opt.beam_size,
    n_best=opt.n_best,
    global_scorer=scorer,
    max_length=opt.max_length,
    copy_attn=model_opt.copy_attn,
    cuda=opt.cuda,
    beam_trace=opt.dump_beam != "",
    min_length=opt.min_length,
    stepwise_penalty=opt.stepwise_penalty)
builder = onmt.translate.TranslationBuilder(
    data, translator.fields,
    opt.n_best, opt.replace_unk, opt.tgt)

# Statistics
counter = count(1)
pred_score_total, pred_words_total = 0, 0
gold_score_total, gold_words_total = 0, 0
i=0

use gates =  False
Loading model parameters.


/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:230: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_1)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:232: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_2)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:234: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_3)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:236: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.H_4)
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/my_modules/GCN.py:41: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_norma

average src size 8.116402116402117 2268


In [30]:
batch=next(iter(data_iter))
"""
Translate a batch of sentences.

Mostly a wrapper around :obj:`Beam`.

Args:
    batch (:obj:`Batch`): a batch from a dataset object
    data (:obj:`Dataset`): the dataset object


Todo:
    Shouldn't need the original dataset.
"""

# (0) Prep each of the components of the search.
# And helper method for reducing verbosity.
beam_size = translator.beam_size
batch_size = batch.batch_size
data_type = data.data_type
vocab = translator.fields["tgt"].vocab
beam = [onmt.translate.Beam(beam_size, n_best=translator.n_best,
                            cuda=translator.cuda,
                            global_scorer=translator.global_scorer,
                            pad=vocab.stoi[onmt.io.PAD_WORD],
                            eos=vocab.stoi[onmt.io.EOS_WORD],
                            bos=vocab.stoi[onmt.io.BOS_WORD],
                            min_length=translator.min_length,
                            stepwise_penalty=translator.stepwise_penalty)
        for __ in range(batch_size)]

# Help functions for working with beams and batches
def var(a): return Variable(a, volatile=True)

def rvar(a): return var(a.repeat(1, beam_size, 1))

def bottle(m):
    return m.view(batch_size * beam_size, -1)

def unbottle(m):
    return m.view(beam_size, batch_size, -1)

# (1) Run the encoder on the src.
src = onmt.io.make_features(batch, 'src', data_type)
for sent in src:
    for word_id in range(len(sent.view(-1))-1):
        print(fields["src"].vocab.itos[word_id],end=' ')
    print('\n')
src_lengths = None
if data_type == 'text':
    _, src_lengths = batch.src
elif data_type == 'gcn':
    _, src_lengths = batch.src
    # report_stats.n_src_words += src_lengths.sum()
    adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
    mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
    if hasattr(batch, 'morph'):
        morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

if data_type == 'gcn':
    # F-prop through the model.
    if hasattr(batch, 'morph'):
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                                adj_arc_in, adj_arc_out, adj_lab_in,
                                adj_lab_out, mask_in, mask_out,
                                mask_loop, mask_sent, morph, mask_morph)
    else:
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                        adj_arc_in, adj_arc_out, adj_lab_in,
                        adj_lab_out, mask_in, mask_out,
                        mask_loop, mask_sent)
else:
    enc_states, memory_bank = translator.model.encoder(src, src_lengths)

# print("memory_bank:",str(memory_bank))
# print("memory_bank.data:",str(memory_bank.data))
# print("size memory_bank:",str(memory_bank.size()))
# print("size memory_bank.data:",str(memory_bank.size()))

dec_states = translator.model.decoder.init_decoder_state(
                                src, memory_bank, enc_states)

if src_lengths is None:
    src_lengths = torch.Tensor(batch_size).type_as(memory_bank.data)\
                                            .long()\
                                            .fill_(memory_bank.size(0))

# (2) Repeat src objects `beam_size` times.
src_map = rvar(batch.src_map.data) \
    if (data_type == 'text' or data_type == 'gcn') and translator.copy_attn else None
memory_bank = rvar(memory_bank.data)
memory_lengths = src_lengths.repeat(beam_size)
dec_states.repeat_beam_size_times(beam_size)

# (3) run the decoder to generate sentences, using beam search.
for i in range(translator.max_length):
    if all((b.done() for b in beam)):
        break

    # Construct batch x beam_size nxt words.
    # Get all the pending current beam words and arrange for forward.
    # print("beam:",beam)
    # print("len beam:", str(len(beam)))
    inp = var(torch.stack([b.get_current_state() for b in beam])
                .t().contiguous().view(1, -1))
    # for b in beam:
        # print("b.currentState:",str(b.get_current_state()), end=' ')
    # inp = rvar(inp)
    # print("size inp:",str(inp.size()))
    # inp = inp.repeat(1,beam_size)
    # print("size inp:",str(inp.size()))


    # Turn any copied words to UNKs
    # 0 is unk
    if translator.copy_attn:
        inp = inp.masked_fill(
            inp.gt(len(translator.fields["tgt"].vocab) - 1), 0)

    # Temporary kludge solution to handle changed dim expectation
    # in the decoder
    inp = inp.unsqueeze(2)

    # Run one step.
    # print("inp:",str(inp))
    # print("size inp:",str(inp.size()))
    # print("memorybank;",str(memory_bank))
    # print("size memorybank:",str(memory_bank.size()))
    dec_out, dec_states, attn = translator.model.decoder(
        inp, memory_bank, dec_states, memory_lengths=memory_lengths)
    dec_out = dec_out.squeeze(0)
    # print("attn",str(attn))
    # print("attn['copy']",str(attn["std"]))
    # print("size attn['copy']",str(attn["std"].size()))
    # dec_out: beam x rnn_size

    # (b) Compute a vector of batch x beam word scores.
    if not translator.copy_attn:
        out = translator.model.generator.forward(dec_out).data
        out = unbottle(out)
        # beam x tgt_vocab
        beam_attn = unbottle(attn["std"])
    else:
        out = translator.model.generator.forward(dec_out,
                                            attn["copy"].squeeze(0),
                                            src_map)
        # beam x (tgt_vocab + extra_vocab)
        out = data.collapse_copy_scores(
            unbottle(out.data),
            batch, translator.fields["tgt"].vocab, data.src_vocabs)
        # beam x tgt_vocab
        out = out.log()
        # print("attn[copy]",str(attn["copy"]))
        beam_attn = unbottle(attn["copy"])
        # print("beam_attn",str(beam_attn))
        # print("size beam_attn",str(beam_attn.size()))
    # (c) Advance each beam.
    for j, b in enumerate(beam):
        advance(b,out[:, j],
                    beam_attn.data[:, j, :memory_lengths[j]])
        dec_states.beam_update(j, b.get_current_origin(), beam_size)
    # j=0
    # b=beam[0]
    # advance(b,out[:, j],
    #             beam_attn.data[:, j, :memory_lengths[j]])
    # dec_states.beam_update(j, b.get_current_origin(), beam_size)

# (4) Extract sentences from beam.
ret = _from_beam(translator,beam)
ret["gold_score"] = [0] * batch_size
if "tgt" in batch.__dict__:
    ret["gold_score"] = translator._run_target(batch, data)
ret["batch"] = batch

/usr/local/lib/python3.7/dist-packages/torchtext/legacy/data/field.py:353: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:908: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  for e in self._all]
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.7/dist-packages/

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 



In [ ]:
src = onmt.io.make_features(batch,'src','gcn')

In [ ]:
src

In [ ]:
for sent in src:
    for word_id in range(len(sent.view(-1))-1):
        print(fields["src"].vocab.itos[word_id],end=' ')
    print('\n')

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 



## Batch Loop

In [31]:
# batch=next(iter(data_iter))
for i, batch in enumerate(data_iter):
    """
    Translate a batch of sentences.

    Mostly a wrapper around :obj:`Beam`.

    Args:
        batch (:obj:`Batch`): a batch from a dataset object
        data (:obj:`Dataset`): the dataset object


    Todo:
        Shouldn't need the original dataset.
    """

    # (0) Prep each of the components of the search.
    # And helper method for reducing verbosity.
    beam_size = translator.beam_size
    batch_size = batch.batch_size
    data_type = data.data_type
    vocab = translator.fields["tgt"].vocab
    beam = [onmt.translate.Beam(beam_size, n_best=translator.n_best,
                                cuda=translator.cuda,
                                global_scorer=translator.global_scorer,
                                pad=vocab.stoi[onmt.io.PAD_WORD],
                                eos=vocab.stoi[onmt.io.EOS_WORD],
                                bos=vocab.stoi[onmt.io.BOS_WORD],
                                min_length=translator.min_length,
                                stepwise_penalty=translator.stepwise_penalty)
            for __ in range(batch_size)]

    # Help functions for working with beams and batches
    def var(a): return Variable(a, volatile=True)

    def rvar(a): return var(a.repeat(1, beam_size, 1))

    def bottle(m):
        return m.view(batch_size * beam_size, -1)

    def unbottle(m):
        return m.view(beam_size, batch_size, -1)

    # (1) Run the encoder on the src.
    src = onmt.io.make_features(batch, 'src', data_type)
    for sent in src:
        for word_id in range(len(sent.view(-1))-1):
            print(fields["src"].vocab.itos[word_id],end=' ')
        print('\n')
    src_lengths = None
    if data_type == 'text':
        _, src_lengths = batch.src
    elif data_type == 'gcn':
        _, src_lengths = batch.src
        # report_stats.n_src_words += src_lengths.sum()
        adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
        mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
        if hasattr(batch, 'morph'):
            morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

    if data_type == 'gcn':
        # F-prop through the model.
        if hasattr(batch, 'morph'):
            enc_states, memory_bank = \
                translator.model.encoder(src, src_lengths,
                                    adj_arc_in, adj_arc_out, adj_lab_in,
                                    adj_lab_out, mask_in, mask_out,
                                    mask_loop, mask_sent, morph, mask_morph)
        else:
            enc_states, memory_bank = \
                translator.model.encoder(src, src_lengths,
                            adj_arc_in, adj_arc_out, adj_lab_in,
                            adj_lab_out, mask_in, mask_out,
                            mask_loop, mask_sent)
    else:
        enc_states, memory_bank = translator.model.encoder(src, src_lengths)

    # print("memory_bank:",str(memory_bank))
    # print("memory_bank.data:",str(memory_bank.data))
    # print("size memory_bank:",str(memory_bank.size()))
    # print("size memory_bank.data:",str(memory_bank.size()))

    dec_states = translator.model.decoder.init_decoder_state(
                                    src, memory_bank, enc_states)

    if src_lengths is None:
        src_lengths = torch.Tensor(batch_size).type_as(memory_bank.data)\
                                                .long()\
                                                .fill_(memory_bank.size(0))

    # (2) Repeat src objects `beam_size` times.
    src_map = rvar(batch.src_map.data) \
        if (data_type == 'text' or data_type == 'gcn') and translator.copy_attn else None
    memory_bank = rvar(memory_bank.data)
    memory_lengths = src_lengths.repeat(beam_size)
    dec_states.repeat_beam_size_times(beam_size)

    # (3) run the decoder to generate sentences, using beam search.
    for i in range(translator.max_length):
        if all((b.done() for b in beam)):
            break

        # Construct batch x beam_size nxt words.
        # Get all the pending current beam words and arrange for forward.
        # print("beam:",beam)
        # print("len beam:", str(len(beam)))
        inp = var(torch.stack([b.get_current_state() for b in beam])
                    .t().contiguous().view(1, -1))
        # for b in beam:
            # print("b.currentState:",str(b.get_current_state()), end=' ')
        # inp = rvar(inp)
        # print("size inp:",str(inp.size()))
        # inp = inp.repeat(1,beam_size)
        # print("size inp:",str(inp.size()))


        # Turn any copied words to UNKs
        # 0 is unk
        if translator.copy_attn:
            inp = inp.masked_fill(
                inp.gt(len(translator.fields["tgt"].vocab) - 1), 0)

        # Temporary kludge solution to handle changed dim expectation
        # in the decoder
        inp = inp.unsqueeze(2)

        # Run one step.
        # print("inp:",str(inp))
        # print("size inp:",str(inp.size()))
        # print("memorybank;",str(memory_bank))
        # print("size memorybank:",str(memory_bank.size()))
        dec_out, dec_states, attn = translator.model.decoder(
            inp, memory_bank, dec_states, memory_lengths=memory_lengths)
        dec_out = dec_out.squeeze(0)
        # print("attn",str(attn))
        # print("attn['copy']",str(attn["std"]))
        # print("size attn['copy']",str(attn["std"].size()))
        # dec_out: beam x rnn_size

        # (b) Compute a vector of batch x beam word scores.
        if not translator.copy_attn:
            out = translator.model.generator.forward(dec_out).data
            out = unbottle(out)
            # beam x tgt_vocab
            beam_attn = unbottle(attn["std"])
        else:
            out = translator.model.generator.forward(dec_out,
                                                attn["copy"].squeeze(0),
                                                src_map)
            # beam x (tgt_vocab + extra_vocab)
            out = data.collapse_copy_scores(
                unbottle(out.data),
                batch, translator.fields["tgt"].vocab, data.src_vocabs)
            # beam x tgt_vocab
            out = out.log()
            # print("attn[copy]",str(attn["copy"]))
            beam_attn = unbottle(attn["copy"])
            # print("beam_attn",str(beam_attn))
            # print("size beam_attn",str(beam_attn.size()))
        # (c) Advance each beam.
        for j, b in enumerate(beam):
            advance(b,out[:, j],
                        beam_attn.data[:, j, :memory_lengths[j]])
            dec_states.beam_update(j, b.get_current_origin(), beam_size)
        # j=0
        # b=beam[0]
        # advance(b,out[:, j],
        #             beam_attn.data[:, j, :memory_lengths[j]])
        # dec_states.beam_update(j, b.get_current_origin(), beam_size)

    # (4) Extract sentences from beam.
    ret = _from_beam(translator,beam)
    ret["gold_score"] = [0] * batch_size
    if "tgt" in batch.__dict__:
        ret["gold_score"] = translator._run_target(batch, data)
    ret["batch"] = batch

/usr/local/lib/python3.7/dist-packages/torchtext/legacy/data/field.py:353: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/Models.py:908: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  for e in self._all]
/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text/onmt/modules/GlobalAttention.py:176: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.7/dist-packages/

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingredient region city and 0 di ispartof 

<unk> <blank> . , country of states airport - united location ( ) leadername language the a international new university c county ingre

In [ ]:
# Batch loop
"""
Translate a batch of sentences.

Mostly a wrapper around :obj:`Beam`.

Args:
    batch (:obj:`Batch`): a batch from a dataset object
    data (:obj:`Dataset`): the dataset object


Todo:
    Shouldn't need the original dataset.
"""

"""
Translate a batch of sentences.

Mostly a wrapper around :obj:`Beam`.

Args:
    batch (:obj:`Batch`): a batch from a dataset object
    data (:obj:`Dataset`): the dataset object


Todo:
    Shouldn't need the original dataset.
"""
for i, batch in enumerate(data_iter):
    if i>=1:
        break
    # (0) Prep each of the components of the search.
    # And helper method for reducing verbosity.
    beam_size = translator.beam_size
    batch_size = batch.batch_size
    data_type = data.data_type
    vocab = translator.fields["tgt"].vocab
    beam = [onmt.translate.Beam(beam_size, n_best=translator.n_best,
                                cuda=translator.cuda,
                                global_scorer=translator.global_scorer,
                                pad=vocab.stoi[onmt.io.PAD_WORD],
                                eos=vocab.stoi[onmt.io.EOS_WORD],
                                bos=vocab.stoi[onmt.io.BOS_WORD],
                                min_length=translator.min_length,
                                stepwise_penalty=translator.stepwise_penalty)
            for __ in range(batch_size)]

    # Help functions for working with beams and batches
    def var(a): return Variable(a, volatile=True)

    def rvar(a): return var(a.repeat(1, beam_size, 1))

    def bottle(m):
        return m.view(batch_size * beam_size, -1)

    def unbottle(m):
        return m.view(beam_size, batch_size, -1)

    # (1) Run the encoder on the src.
    src = onmt.io.make_features(batch, 'src', data_type)
    src_lengths = None
    if data_type == 'text':
        _, src_lengths = batch.src
    elif data_type == 'gcn':
        _, src_lengths = batch.src
        # report_stats.n_src_words += src_lengths.sum()
        adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
        mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
        if hasattr(batch, 'morph'):
            morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

    if data_type == 'gcn':
        # F-prop through the model.
        if hasattr(batch, 'morph'):
            enc_states, memory_bank = \
                translator.model.encoder(src, src_lengths,
                                    adj_arc_in, adj_arc_out, adj_lab_in,
                                    adj_lab_out, mask_in, mask_out,
                                    mask_loop, mask_sent, morph, mask_morph)
        else:
            enc_states, memory_bank = \
                translator.model.encoder(src, src_lengths,
                            adj_arc_in, adj_arc_out, adj_lab_in,
                            adj_lab_out, mask_in, mask_out,
                            mask_loop, mask_sent)
    else:
        enc_states, memory_bank = translator.model.encoder(src, src_lengths)

    # print("memory_bank:",str(memory_bank))
    # print("memory_bank.data:",str(memory_bank.data))
    # print("size memory_bank:",str(memory_bank.size()))
    # print("size memory_bank.data:",str(memory_bank.size()))

    dec_states = translator.model.decoder.init_decoder_state(
                                    src, memory_bank, enc_states)

    if src_lengths is None:
        src_lengths = torch.Tensor(batch_size).type_as(memory_bank.data)\
                                                .long()\
                                                .fill_(memory_bank.size(0))

    # (2) Repeat src objects `beam_size` times.
    src_map = rvar(batch.src_map.data) \
        if (data_type == 'text' or data_type == 'gcn') and translator.copy_attn else None
    memory_bank = rvar(memory_bank.data)
    memory_lengths = src_lengths.repeat(beam_size)
    dec_states.repeat_beam_size_times(beam_size)

    # (3) run the decoder to generate sentences, using beam search.
    for i in range(translator.max_length):
        if all((b.done() for b in beam)):
            break

        # Construct batch x beam_size nxt words.
        # Get all the pending current beam words and arrange for forward.
        # print("beam:",beam)
        # print("len beam:", str(len(beam)))
        inp = var(torch.stack([b.get_current_state() for b in beam])
                    .t().contiguous().view(1, -1))
        # for b in beam:
            # print("b.currentState:",str(b.get_current_state()), end=' ')
        # inp = rvar(inp)
        # print("size inp:",str(inp.size()))
        # inp = inp.repeat(1,beam_size)
        # print("size inp:",str(inp.size()))


        # Turn any copied words to UNKs
        # 0 is unk
        if translator.copy_attn:
            inp = inp.masked_fill(
                inp.gt(len(translator.fields["tgt"].vocab) - 1), 0)

        # Temporary kludge solution to handle changed dim expectation
        # in the decoder
        inp = inp.unsqueeze(2)

        # Run one step.
        # print("inp:",str(inp))
        # print("size inp:",str(inp.size()))
        # print("memorybank;",str(memory_bank))
        # print("size memorybank:",str(memory_bank.size()))
        dec_out, dec_states, attn = translator.model.decoder(
            inp, memory_bank, dec_states, memory_lengths=memory_lengths)
        dec_out = dec_out.squeeze(0)
        # dec_out: beam x rnn_size

        # (b) Compute a vector of batch x beam word scores.
        if not translator.copy_attn:
            out = translator.model.generator.forward(dec_out).data
            out = unbottle(out)
            # beam x tgt_vocab
            beam_attn = unbottle(attn["std"])
        else:
            out = translator.model.generator.forward(dec_out,
                                                attn["copy"].squeeze(0),
                                                src_map)
            # beam x (tgt_vocab + extra_vocab)
            out = data.collapse_copy_scores(
                unbottle(out.data),
                batch, translator.fields["tgt"].vocab, data.src_vocabs)
            # beam x tgt_vocab
            out = out.log()
            beam_attn = unbottle(attn["copy"])
        # (c) Advance each beam.
        for j, b in enumerate(beam):
            advance(b,out[:, j],
                        beam_attn.data[:, j, :memory_lengths[j]])
            dec_states.beam_update(j, b.get_current_origin(), beam_size)
            print("advance one loop done")

    # (4) Extract sentences from beam.
    ret = _from_beam(translator,beam)
    ret["gold_score"] = [0] * batch_size
    if "tgt" in batch.__dict__:
        ret["gold_score"] = translator._run_target(batch, data)
    ret["batch"] = batch

    print("one batch loop done!")

# Fields

In [ ]:
fields

{'alignment': <torchtext.legacy.data.field.Field at 0x7f3d43595810>,
 'indices': <torchtext.legacy.data.field.Field at 0x7f3d43595690>,
 'label': <torchtext.legacy.data.field.Field at 0x7f3d43595650>,
 'morph': <torchtext.legacy.data.field.Field at 0x7f3d43595450>,
 'node1': <torchtext.legacy.data.field.Field at 0x7f3d43595590>,
 'node2': <torchtext.legacy.data.field.Field at 0x7f3d435953d0>,
 'src': <torchtext.legacy.data.field.Field at 0x7f3d43669c10>,
 'src_map': <torchtext.legacy.data.field.Field at 0x7f3d435958d0>,
 'tgt': <torchtext.legacy.data.field.Field at 0x7f3d43ee4a10>}

In [ ]:
fields["tgt"].vocab.itos[3]

'</s>'

In [ ]:
fields[""]

In [ ]:
ret["predictions"][0][0]

[tensor(5),
 tensor(95),
 tensor(8),
 tensor(5),
 tensor(535),
 tensor(4),
 tensor(3)]

# Translation TEST

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks/Others/graph-2-text

/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text


In [ ]:
from __future__ import division, unicode_literals
import os
import argparse
import math
import codecs
import torch

In [ ]:
from itertools import count

In [ ]:
import onmt.io
import onmt.translate
import onmt
import onmt.ModelConstructor
import onmt.modules
import opts

In [ ]:
parser = argparse.ArgumentParser(
    description='translate.py',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
opts.add_md_help_argument(parser)
opts.translate_opts(parser)

In [ ]:
opt = parser.parse_args("-model data/tmp__acc_69.72_ppl_3.40_e10.pt -data_type gcn -src data/webnlg/dev-webnlg-all-delex-src-nodes.txt -tgt data/webnlg/dev-webnlg-all-delex-tgt.txt -src_label data/webnlg/dev-webnlg-all-delex-src-labels.txt -src_node1 data/webnlg/dev-webnlg-all-delex-src-node1.txt -src_node2 data/webnlg/dev-webnlg-all-delex-src-node2.txt -output data/webnlg/delexicalized_predictions_dev.txt -replace_unk -verbose".split())

In [ ]:
def _report_score(name, score_total, words_total):
    print("%s AVG SCORE: %.4f, %s PPL: %.4f" % (
        name, score_total / words_total,
        name, math.exp(-score_total / words_total)))

In [ ]:
def _report_bleu():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    print()
    res = subprocess.check_output(
        "perl %s/tools/multi-bleu.perl %s < %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(">> " + res.strip())

In [ ]:
def _report_rouge():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    res = subprocess.check_output(
        "python %s/tools/test_rouge.py -r %s -c %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(res.strip())

In [ ]:
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]
opt.cuda = opt.gpu > -1
if opt.cuda:
    # torch.cuda.set_device(opt.gpu)
    torch.cuda.set_device('cuda:0')

# Load the model.
fields, model, model_opt = \
    onmt.ModelConstructor.load_test_model(opt, dummy_opt.__dict__)

# File to write sentences to.
out_file = codecs.open(opt.output, 'w', 'utf-8')
data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                 opt.src, opt.tgt, opt.src_label,
                                 opt.src_node1, opt.src_node2,
                                 opt.src_morph,
                                 src_dir=opt.src_dir,
                                 sample_rate=opt.sample_rate,
                                 window_size=opt.window_size,
                                 window_stride=opt.window_stride,
                                 window=opt.window,
                                 use_filter_pred=False)
data_iter = onmt.io.OrderedIterator(
        dataset=data, device=opt.gpu,
        batch_size=opt.batch_size, train=False, sort=False,
        sort_within_batch=True, shuffle=False)
scorer = onmt.translate.GNMTGlobalScorer(opt.alpha,
                                             opt.beta,
                                             opt.coverage_penalty,
                                             opt.length_penalty)
translator = onmt.translate.Translator(
        model, fields,
        beam_size=opt.beam_size,
        n_best=opt.n_best,
        global_scorer=scorer,
        max_length=opt.max_length,
        copy_attn=model_opt.copy_attn,
        cuda=opt.cuda,
        beam_trace=opt.dump_beam != "",
        min_length=opt.min_length,
        stepwise_penalty=opt.stepwise_penalty)


builder = onmt.translate.TranslationBuilder(
        data, translator.fields,
        opt.n_best, opt.replace_unk, opt.tgt)

TypeError: ignored

In [ ]:
from torch.autograd import Variable

In [ ]:
def from_batch(builder, translation_batch):
        batch = translation_batch["batch"]
        assert(len(translation_batch["gold_score"]) ==
               len(translation_batch["predictions"]))
        batch_size = batch.batch_size

        preds, pred_score, attn, gold_score, indices = list(zip(
            *sorted(zip(translation_batch["predictions"],
                        translation_batch["scores"],
                        translation_batch["attention"],
                        translation_batch["gold_score"],
                        batch.indices.data),
                    key=lambda x: x[-1])))

        # Sorting
        inds, perm = torch.sort(batch.indices.data)
        data_type = builder.data.data_type
        if data_type == 'text':
            src = batch.src[0].data.index_select(1, perm)
        else:
            src = None

        if builder.has_tgt:
            tgt = batch.tgt.data.index_select(1, perm)
        else:
            tgt = None

        translations = []
        for b in range(batch_size):
            if data_type == 'text' or data_type == 'gcn':
                src_vocab = builder.data.src_vocabs[inds[b]] \
                  if builder.data.src_vocabs else None
                src_raw = builder.data.examples[inds[b]].src
            else:
                src_vocab = None
                src_raw = None
            pred_sents = [_build_target_tokens(
                builder,
                src[:, b] if src is not None else None,
                src_vocab, src_raw,
                preds[b][n], attn[b][n])
                          for n in range(builder.n_best)]
            gold_sent = None
            if tgt is not None:
                gold_sent = _build_target_tokens(
                    builder,
                    src[:, b] if src is not None else None,
                    src_vocab, src_raw,
                    tgt[1:, b] if tgt is not None else None, None)

            translation = onmt.translate.Translation(src[:, b] if src is not None else None,
                                      src_raw, pred_sents,
                                      attn[b], pred_score[b], gold_sent,
                                      gold_score[b])
            translations.append(translation)

        return translations

In [ ]:
def _build_target_tokens(builder, src, src_vocab, src_raw, pred, attn):
        vocab = builder.fields["tgt"].vocab
        tokens = []
        for tok in pred:
            if tok < len(vocab):
                # print("tok",str(tok))
                # print("type tok", str(type(tok)))
                # print("type tok",str(tok.type()))
                # print("type tok", str(tok.type(torch.int64)))
                tokens.append(vocab.itos[tok.type(torch.int64)])
                # print("tok:",vocab.itos[tok.type(torch.int64)])
            else:
                tokens.append(src_vocab.itos[tok - len(vocab)])
            if tokens[-1] == onmt.io.EOS_WORD:
                tokens = tokens[:-1]
                break
        if builder.replace_unk and (attn is not None) and (src is not None):
            for i in range(len(tokens)):
                if tokens[i] == vocab.itos[onmt.io.UNK]:
                    _, maxIndex = attn[i].max(0)
                    tokens[i] = src_raw[maxIndex[0]]

        return tokens

In [ ]:
vocab = builder.fields["tgt"].vocab

In [ ]:
len(vocab.freqs)

4634

## from batch TEST

In [ ]:
i=0
batch=data_iter.__iter__().__next__()
batch_data = translator.translate_batch(batch, data)

In [ ]:
# from_batch TEST
translation_batch = batch_data
batch = translation_batch["batch"]
assert(len(translation_batch["gold_score"]) ==
        len(translation_batch["predictions"]))
batch_size = batch.batch_size

preds, pred_score, attn, gold_score, indices = list(zip(
    *sorted(zip(translation_batch["predictions"],
                translation_batch["scores"],
                translation_batch["attention"],
                translation_batch["gold_score"],
                batch.indices.data),
            key=lambda x: x[-1])))

print("preds",str(preds))
print("size preds",str(len(preds)))

# Sorting
inds, perm = torch.sort(batch.indices.data)
data_type = builder.data.data_type
if data_type == 'text':
    src = batch.src[0].data.index_select(1, perm)
else:
    src = None

if builder.has_tgt:
    tgt = batch.tgt.data.index_select(1, perm)
else:
    tgt = None

translations = []
for b in range(batch_size):
    if data_type == 'text' or data_type == 'gcn':
        src_vocab = builder.data.src_vocabs[inds[b]] \
            if builder.data.src_vocabs else None
        src_raw = builder.data.examples[inds[b]].src
    else:
        src_vocab = None
        src_raw = None
    pred_sents = [_build_target_tokens(
        builder,
        src[:, b] if src is not None else None,
        src_vocab, src_raw,
        preds[b][n], attn[b][n])
                    for n in range(builder.n_best)]
    gold_sent = None
    if tgt is not None:
        gold_sent = _build_target_tokens(
            builder,
            src[:, b] if src is not None else None,
            src_vocab, src_raw,
            tgt[1:, b] if tgt is not None else None, None)

    translation = onmt.translate.Translation(src[:, b] if src is not None else None,
                                src_raw, pred_sents,
                                attn[b], pred_score[b], gold_sent,
                                gold_score[b])
    translations.append(translation)

preds ([[tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tensor(0., device='cuda:0'), tenso

## _build_target_tokens

In [ ]:
# _build_target_tokens
src = src[:, b] if src is not None else None,
src_vocab = src_vocab
src_raw = src_raw
pred=preds[b][0]
vocab = builder.fields["tgt"].vocab
print("vocab",str(vocab))
tokens = []
for tok in pred:
    if tok < len(vocab):
        # print("tok",str(tok))
        # print("type tok", str(type(tok)))
        # print("type tok",str(tok.type()))
        # print("type tok", str(tok.type(torch.int64)))
        tokens.append(vocab.itos[tok.type(torch.int64)])
        # print("tok:",vocab.itos[tok.type(torch.int64)])
    else:
        tokens.append(src_vocab.itos[tok - len(vocab)])
    if tokens[-1] == onmt.io.EOS_WORD:
        tokens = tokens[:-1]
        break
if builder.replace_unk and (attn is not None) and (src is not None):
    for i in range(len(tokens)):
        if tokens[i] == vocab.itos[onmt.io.UNK]:
            _, maxIndex = attn[i].max(0)
            tokens[i] = src_raw[maxIndex[0]]

TypeError: ignored

In [ ]:
print(src_vocab.itos[preds[0] - len(vocab)])

NameError: ignored

In [ ]:
i=0
batch=data_iter.__iter__().__next__()
batch_data = translator.translate_batch(batch, data)
translations = from_batch(builder,batch_data)


In [ ]:
test = []
test.append(torch.tensor(5))

In [ ]:
vocab = builder.fields["tgt"].vocab

In [ ]:
vocab.itos[50]

'its'

# Translator_batch TEST in Translator.py

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks/Others/graph-2-text

In [ ]:
from __future__ import division, unicode_literals
import os
import argparse
import math
import codecs
import torch

In [ ]:
from itertools import count

In [ ]:
import onmt.io
import onmt.translate
import onmt
import onmt.ModelConstructor
import onmt.modules
import opts

In [ ]:
parser = argparse.ArgumentParser(
    description='translate.py',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
opts.add_md_help_argument(parser)
opts.translate_opts(parser)

In [ ]:
opt = parser.parse_args("-model data/tmp__acc_70.16_ppl_3.33_e10.pt -data_type gcn -src data/webnlg/dev-webnlg-all-delex-src-nodes.txt -tgt data/webnlg/dev-webnlg-all-delex-tgt.txt -src_label data/webnlg/dev-webnlg-all-delex-src-labels.txt -src_node1 data/webnlg/dev-webnlg-all-delex-src-node1.txt -src_node2 data/webnlg/dev-webnlg-all-delex-src-node2.txt -output data/webnlg/delexicalized_predictions_dev.txt -replace_unk -verbose".split())

In [ ]:
def _report_score(name, score_total, words_total):
    print("%s AVG SCORE: %.4f, %s PPL: %.4f" % (
        name, score_total / words_total,
        name, math.exp(-score_total / words_total)))

In [ ]:
def _report_bleu():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    print()
    res = subprocess.check_output(
        "perl %s/tools/multi-bleu.perl %s < %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(">> " + res.strip())

In [ ]:
def _report_rouge():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    res = subprocess.check_output(
        "python %s/tools/test_rouge.py -r %s -c %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(res.strip())

In [ ]:
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]
opt.cuda = opt.gpu > -1
if opt.cuda:
    # torch.cuda.set_device(opt.gpu)
    torch.cuda.set_device('cuda:0')

# Load the model.
fields, model, model_opt = \
    onmt.ModelConstructor.load_test_model(opt, dummy_opt.__dict__)

# File to write sentences to.
out_file = codecs.open(opt.output, 'w', 'utf-8')
data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                 opt.src, opt.tgt, opt.src_label,
                                 opt.src_node1, opt.src_node2,
                                 opt.src_morph,
                                 src_dir=opt.src_dir,
                                 sample_rate=opt.sample_rate,
                                 window_size=opt.window_size,
                                 window_stride=opt.window_stride,
                                 window=opt.window,
                                 use_filter_pred=False)
data_iter = onmt.io.OrderedIterator(
        dataset=data, device=opt.gpu,
        batch_size=opt.batch_size, train=False, sort=False,
        sort_within_batch=True, shuffle=False)
scorer = onmt.translate.GNMTGlobalScorer(opt.alpha,
                                             opt.beta,
                                             opt.coverage_penalty,
                                             opt.length_penalty)
translator = onmt.translate.Translator(
        model, fields,
        beam_size=opt.beam_size,
        n_best=opt.n_best,
        global_scorer=scorer,
        max_length=opt.max_length,
        copy_attn=model_opt.copy_attn,
        cuda=opt.cuda,
        beam_trace=opt.dump_beam != "",
        min_length=opt.min_length,
        stepwise_penalty=opt.stepwise_penalty)

In [ ]:
for i, batch in enumerate(data_iter):
    print(str(i), str(batch.batch_size))
    beam_size = translator.beam_size
    batch_size = batch.batch_size
    data_type = data.data_type
    vocab = translator.fields["tgt"].vocab
    beam = []
    for __ in range(batch.batch_size):
        beam.append(onmt.translate.Beam(beam_size, n_best=translator.n_best,
                                    cuda=translator.cuda,
                                    global_scorer=translator.global_scorer,
                                    pad=vocab.stoi[onmt.io.PAD_WORD],
                                    eos=vocab.stoi[onmt.io.EOS_WORD],
                                    bos=vocab.stoi[onmt.io.BOS_WORD],
                                    min_length=translator.min_length,
                                    stepwise_penalty=translator.stepwise_penalty))

In [ ]:
from torch.autograd import Variable
def var(a): return Variable(a, volatile=True)
inp = var(torch.stack([b.get_current_state() for b in beam])
                      .t().contiguous().view(1, -1))

In [ ]:
"""
Translate a batch of sentences.

Mostly a wrapper around :obj:`Beam`.

Args:
    batch (:obj:`Batch`): a batch from a dataset object
    data (:obj:`Dataset`): the dataset object


Todo:
    Shouldn't need the original dataset.
"""

# (0) Prep each of the components of the search.
# And helper method for reducing verbosity.
beam_size = translator.beam_size
batch_size = batch.batch_size
data_type = data.data_type
vocab = translator.fields["tgt"].vocab
beam = [onmt.translate.Beam(beam_size, n_best=translator.n_best,
                            cuda=translator.cuda,
                            global_scorer=translator.global_scorer,
                            pad=vocab.stoi[onmt.io.PAD_WORD],
                            eos=vocab.stoi[onmt.io.EOS_WORD],
                            bos=vocab.stoi[onmt.io.BOS_WORD],
                            min_length=translator.min_length,
                            stepwise_penalty=translator.stepwise_penalty)
        for __ in range(batch_size)]

# Help functions for working with beams and batches
def var(a): return Variable(a, volatile=True)

def rvar(a): return var(a.repeat(1, beam_size, 1))

def bottle(m):
    return m.view(batch_size * beam_size, -1)

def unbottle(m):
    return m.view(beam_size, batch_size, -1)

# (1) Run the encoder on the src.
src = onmt.io.make_features(batch, 'src', data_type)
src_lengths = None
if data_type == 'text':
    _, src_lengths = batch.src
elif data_type == 'gcn':
    _, src_lengths = batch.src
    # report_stats.n_src_words += src_lengths.sum()
    adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
    mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
    if hasattr(batch, 'morph'):
        morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

if data_type == 'gcn':
    # F-prop through the model.
    if hasattr(batch, 'morph'):
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                                adj_arc_in, adj_arc_out, adj_lab_in,
                                adj_lab_out, mask_in, mask_out,
                                mask_loop, mask_sent, morph, mask_morph)
    else:
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                        adj_arc_in, adj_arc_out, adj_lab_in,
                        adj_lab_out, mask_in, mask_out,
                        mask_loop, mask_sent)
else:
    enc_states, memory_bank = translator.model.encoder(src, src_lengths)

# print("memory_bank:",str(memory_bank))
# print("memory_bank.data:",str(memory_bank.data))
# print("size memory_bank:",str(memory_bank.size()))
# print("size memory_bank.data:",str(memory_bank.size()))

dec_states = translator.model.decoder.init_decoder_state(
                                src, memory_bank, enc_states)

if src_lengths is None:
    src_lengths = torch.Tensor(batch_size).type_as(memory_bank.data)\
                                            .long()\
                                            .fill_(memory_bank.size(0))

# (2) Repeat src objects `beam_size` times.
src_map = rvar(batch.src_map.data) \
    if (data_type == 'text' or data_type == 'gcn') and translator.copy_attn else None
memory_bank = rvar(memory_bank.data)
memory_lengths = src_lengths.repeat(beam_size)
dec_states.repeat_beam_size_times(beam_size)

# (3) run the decoder to generate sentences, using beam search.
for i in range(translator.max_length):
    if all((b.done() for b in beam)):
        break

    # Construct batch x beam_size nxt words.
    # Get all the pending current beam words and arrange for forward.
    print("beam:",beam)
    print("len beam:", str(len(beam)))
    inp = var(torch.stack([b.get_current_state() for b in beam])
                .t().contiguous().view(1, -1))
    print("var(torch.stack([b.get_current_state() for b in beam]))",str(var(torch.stack([b.get_current_state() for b in beam]))))
    for b in beam:
        print(str(b),str(b.get_current_state()))
    # print("torch.stack.t().contiguous().view(1,-1)",str(var(torch.stack([b.get_current_state() for b in beam])
    #             .t().contiguous().view(1, -1))))
    # print("size var(torch.stack([b.get_current_state() for b in beam]))",str(var(torch.stack([b.get_current_state() for b in beam])).size()))
    # print("size torch.stack.t().contiguous().view(1,-1)",str(var(torch.stack([b.get_current_state() for b in beam])
                # .t().contiguous().view(1, -1)).size()))
    # inp = rvar(inp)
    print("size inp:",str(inp.size()))
    # inp = inp.repeat(1,5)
    # print("size inp:",str(inp.size()))


    # Turn any copied words to UNKs
    # 0 is unk
    if translator.copy_attn:
        inp = inp.masked_fill(
            inp.gt(len(translator.fields["tgt"].vocab) - 1), 0)

    # Temporary kludge solution to handle changed dim expectation
    # in the decoder
    inp = inp.unsqueeze(2)

    # Run one step.
    # print("inp:",str(inp))
    # print("size inp:",str(inp.size()))
    # print("memorybank;",str(memory_bank))
    # print("size memorybank:",str(memory_bank.size()))
    # print("dec_states:",str(dec_states))
    # print("dec_states.input_feed:",str(dec_states.input_feed))
    # print("dec_states.input_feed.squeeze(0):",str(dec_states.input_feed.squeeze(0)))
    # print("dec_states.input_feed.size():",str(dec_states.input_feed.size()))
    # print("dec_states.input_feed.squeeze(0).size():",str(dec_states.input_feed.squeeze(0).size()))
    dec_out, dec_states, attn = translator.model.decoder(
        inp, memory_bank, dec_states, memory_lengths=memory_lengths)
    dec_out = dec_out.squeeze(0)
    # dec_out: beam x rnn_size

    # (b) Compute a vector of batch x beam word scores.
    if not translator.copy_attn:
        out = translator.model.generator.forward(dec_out).data
        out = unbottle(out)
        # beam x tgt_vocab
        beam_attn = unbottle(attn["std"])
    else:
        out = translator.model.generator.forward(dec_out,
                                            attn["copy"].squeeze(0),
                                            src_map)
        # beam x (tgt_vocab + extra_vocab)
        out = data.collapse_copy_scores(
            unbottle(out.data),
            batch, translator.fields["tgt"].vocab, data.src_vocabs)
        # beam x tgt_vocab
        out = out.log()
        beam_attn = unbottle(attn["copy"])
    # (c) Advance each beam.
    for j, b in enumerate(beam):
        b.advance(out[:, j],
                    beam_attn.data[:, j, :memory_lengths[j]])
        dec_states.beam_update(j, b.get_current_origin(), beam_size)

# (4) Extract sentences from beam.
ret = translator._from_beam(beam)
ret["gold_score"] = [0] * batch_size
if "tgt" in batch.__dict__:
    ret["gold_score"] = translator._run_target(batch, data)
ret["batch"] = batch
return ret

# Translator_batch TEST in Translator.py WITH BATCH

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks/Others/graph-2-text

In [ ]:
from __future__ import division, unicode_literals
import os
import argparse
import math
import codecs
import torch

In [ ]:
from itertools import count

In [ ]:
import onmt.io
import onmt.translate
import onmt
import onmt.ModelConstructor
import onmt.modules
import opts

In [ ]:
parser = argparse.ArgumentParser(
    description='translate.py',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
opts.add_md_help_argument(parser)
opts.translate_opts(parser)

In [ ]:
opt = parser.parse_args("-model data/tmp__acc_70.16_ppl_3.33_e10.pt -data_type gcn -src data/webnlg/dev-webnlg-all-delex-src-nodes.txt -tgt data/webnlg/dev-webnlg-all-delex-tgt.txt -src_label data/webnlg/dev-webnlg-all-delex-src-labels.txt -src_node1 data/webnlg/dev-webnlg-all-delex-src-node1.txt -src_node2 data/webnlg/dev-webnlg-all-delex-src-node2.txt -output data/webnlg/delexicalized_predictions_dev.txt -replace_unk -verbose".split())

In [ ]:
def _report_score(name, score_total, words_total):
    print("%s AVG SCORE: %.4f, %s PPL: %.4f" % (
        name, score_total / words_total,
        name, math.exp(-score_total / words_total)))

In [ ]:
def _report_bleu():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    print()
    res = subprocess.check_output(
        "perl %s/tools/multi-bleu.perl %s < %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(">> " + res.strip())

In [ ]:
def _report_rouge():
    import subprocess
    path = os.path.split(os.path.realpath(__file__))[0]
    res = subprocess.check_output(
        "python %s/tools/test_rouge.py -r %s -c %s"
        % (path, opt.tgt, opt.output),
        shell=True).decode("utf-8")
    print(res.strip())

In [ ]:
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]
opt.cuda = opt.gpu > -1
if opt.cuda:
    # torch.cuda.set_device(opt.gpu)
    torch.cuda.set_device('cuda:0')

# Load the model.
fields, model, model_opt = \
    onmt.ModelConstructor.load_test_model(opt, dummy_opt.__dict__)

# File to write sentences to.
out_file = codecs.open(opt.output, 'w', 'utf-8')
data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                 opt.src, opt.tgt, opt.src_label,
                                 opt.src_node1, opt.src_node2,
                                 opt.src_morph,
                                 src_dir=opt.src_dir,
                                 sample_rate=opt.sample_rate,
                                 window_size=opt.window_size,
                                 window_stride=opt.window_stride,
                                 window=opt.window,
                                 use_filter_pred=False)
data_iter = onmt.io.OrderedIterator(
        dataset=data, device=opt.gpu,
        batch_size=opt.batch_size, train=False, sort=False,
        sort_within_batch=True, shuffle=False)
scorer = onmt.translate.GNMTGlobalScorer(opt.alpha,
                                             opt.beta,
                                             opt.coverage_penalty,
                                             opt.length_penalty)
translator = onmt.translate.Translator(
        model, fields,
        beam_size=opt.beam_size,
        n_best=opt.n_best,
        global_scorer=scorer,
        max_length=opt.max_length,
        copy_attn=model_opt.copy_attn,
        cuda=opt.cuda,
        beam_trace=opt.dump_beam != "",
        min_length=opt.min_length,
        stepwise_penalty=opt.stepwise_penalty)

In [ ]:
from torch.autograd import Variable

In [ ]:
for i, batch in enumerate(data_iter):
    print(str(i), str(batch.batch_size))
    beam_size = translator.beam_size
    batch_size = batch.batch_size
    data_type = data.data_type
    vocab = translator.fields["tgt"].vocab
    beam = []
    for __ in range(batch.batch_size):
        beam_size = translator.beam_size
        batch_size = batch.batch_size
        data_type = data.data_type
        vocab = translator.fields["tgt"].vocab
        beam = [onmt.translate.Beam(beam_size, n_best=translator.n_best,
                                    cuda=translator.cuda,
                                    global_scorer=translator.global_scorer,
                                    pad=vocab.stoi[onmt.io.PAD_WORD],
                                    eos=vocab.stoi[onmt.io.EOS_WORD],
                                    bos=vocab.stoi[onmt.io.BOS_WORD],
                                    min_length=translator.min_length,
                                    stepwise_penalty=translator.stepwise_penalty)
                for __ in range(batch_size)]
        # Help functions for working with beams and batches
        def var(a): return Variable(a, volatile=True)

        def rvar(a): return var(a.repeat(1, beam_size, 1))

        def bottle(m):
            return m.view(batch_size * beam_size, -1)

        def unbottle(m):
            return m.view(beam_size, batch_size, -1)

        # (1) Run the encoder on the src.
        src = onmt.io.make_features(batch, 'src', data_type)
        src_lengths = None
        if data_type == 'text':
            _, src_lengths = batch.src
        elif data_type == 'gcn':
            _, src_lengths = batch.src
            # report_stats.n_src_words += src_lengths.sum()
            adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
            mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
            if hasattr(batch, 'morph'):
                morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]

        if data_type == 'gcn':
            # F-prop through the model.
            if hasattr(batch, 'morph'):
                enc_states, memory_bank = \
                    translator.model.encoder(src, src_lengths,
                                        adj_arc_in, adj_arc_out, adj_lab_in,
                                        adj_lab_out, mask_in, mask_out,
                                        mask_loop, mask_sent, morph, mask_morph)
            else:
                enc_states, memory_bank = \
                    translator.model.encoder(src, src_lengths,
                                adj_arc_in, adj_arc_out, adj_lab_in,
                                adj_lab_out, mask_in, mask_out,
                                mask_loop, mask_sent)
        else:
            enc_states, memory_bank = translator.model.encoder(src, src_lengths)

        # print("memory_bank:",str(memory_bank))
        # print("memory_bank.data:",str(memory_bank.data))
        # print("size memory_bank:",str(memory_bank.size()))
        # print("size memory_bank.data:",str(memory_bank.size()))

        dec_states = translator.model.decoder.init_decoder_state(
                                        src, memory_bank, enc_states)

        if src_lengths is None:
            src_lengths = torch.Tensor(batch_size).type_as(memory_bank.data)\
                                                    .long()\
                                                    .fill_(memory_bank.size(0))

        # (2) Repeat src objects `beam_size` times.
        src_map = rvar(batch.src_map.data) \
            if (data_type == 'text' or data_type == 'gcn') and translator.copy_attn else None
        memory_bank = rvar(memory_bank.data)
        memory_lengths = src_lengths.repeat(beam_size)
        dec_states.repeat_beam_size_times(beam_size)

        # (3) run the decoder to generate sentences, using beam search.
        for i in range(translator.max_length):
            if all((b.done() for b in beam)):
                break

            # Construct batch x beam_size nxt words.
            # Get all the pending current beam words and arrange for forward.
            print("beam:",beam)
            print("len beam:", str(len(beam)))
            inp = var(torch.stack([b.get_current_state() for b in beam])
                        .t().contiguous().view(1, -1))
            print("var(torch.stack([b.get_current_state() for b in beam]))",str(var(torch.stack([b.get_current_state() for b in beam]))))
            for b in beam:
                print(str(b),str(b.get_current_state()))
            # print("torch.stack.t().contiguous().view(1,-1)",str(var(torch.stack([b.get_current_state() for b in beam])
            #             .t().contiguous().view(1, -1))))
            # print("size var(torch.stack([b.get_current_state() for b in beam]))",str(var(torch.stack([b.get_current_state() for b in beam])).size()))
            # print("size torch.stack.t().contiguous().view(1,-1)",str(var(torch.stack([b.get_current_state() for b in beam])
                        # .t().contiguous().view(1, -1)).size()))
            # inp = rvar(inp)
            print("size inp:",str(inp.size()))
            # inp = inp.repeat(1,5)
            # print("size inp:",str(inp.size()))


            # Turn any copied words to UNKs
            # 0 is unk
            if translator.copy_attn:
                inp = inp.masked_fill(
                    inp.gt(len(translator.fields["tgt"].vocab) - 1), 0)

            # Temporary kludge solution to handle changed dim expectation
            # in the decoder
            inp = inp.unsqueeze(2)

            # Run one step.
            # print("inp:",str(inp))
            # print("size inp:",str(inp.size()))
            # print("memorybank;",str(memory_bank))
            # print("size memorybank:",str(memory_bank.size()))
            # print("dec_states:",str(dec_states))
            # print("dec_states.input_feed:",str(dec_states.input_feed))
            # print("dec_states.input_feed.squeeze(0):",str(dec_states.input_feed.squeeze(0)))
            # print("dec_states.input_feed.size():",str(dec_states.input_feed.size()))
            # print("dec_states.input_feed.squeeze(0).size():",str(dec_states.input_feed.squeeze(0).size()))
            dec_out, dec_states, attn = translator.model.decoder(
                inp, memory_bank, dec_states, memory_lengths=memory_lengths)
            dec_out = dec_out.squeeze(0)
            # dec_out: beam x rnn_size

            # (b) Compute a vector of batch x beam word scores.
            if not translator.copy_attn:
                out = translator.model.generator.forward(dec_out).data
                out = unbottle(out)
                # beam x tgt_vocab
                beam_attn = unbottle(attn["std"])
            else:
                out = translator.model.generator.forward(dec_out,
                                                    attn["copy"].squeeze(0),
                                                    src_map)
                # beam x (tgt_vocab + extra_vocab)
                out = data.collapse_copy_scores(
                    unbottle(out.data),
                    batch, translator.fields["tgt"].vocab, data.src_vocabs)
                # beam x tgt_vocab
                out = out.log()
                beam_attn = unbottle(attn["copy"])
            # (c) Advance each beam.
            for j, b in enumerate(beam):
                b.advance(out[:, j],
                            beam_attn.data[:, j, :memory_lengths[j]])
                dec_states.beam_update(j, b.get_current_origin(), beam_size)

        # (4) Extract sentences from beam.
        ret = translator._from_beam(beam)
        ret["gold_score"] = [0] * batch_size
        if "tgt" in batch.__dict__:
            ret["gold_score"] = _run_target(translator,batch, data)
        ret["batch"] = batch


In [ ]:
def _run_target(self, batch, data):
        data_type = data.data_type
        if data_type == 'text':
            _, src_lengths = batch.src
        elif data_type == 'gcn':
            _, src_lengths = batch.src
            # report_stats.n_src_words += src_lengths.sum()
            adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
            mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
            if hasattr(batch, 'morph'):
                morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]
        else:
            src_lengths = None
        src = onmt.io.make_features(batch, 'src', data_type)
        tgt_in = onmt.io.make_features(batch, 'tgt')[:-1]



        #  (1) run the encoder on the src
        if data_type == 'gcn':
            # F-prop through the model.
            if hasattr(batch, 'morph'):
                enc_states, memory_bank = \
                    self.model.encoder(src, src_lengths,
                                       adj_arc_in, adj_arc_out, adj_lab_in,
                                       adj_lab_out, mask_in, mask_out,
                                       mask_loop, mask_sent, morph, mask_morph)
            else:
                enc_states, memory_bank = \
                    self.model.encoder(src, src_lengths,
                               adj_arc_in, adj_arc_out, adj_lab_in,
                               adj_lab_out, mask_in, mask_out,
                               mask_loop, mask_sent)

        else:
            enc_states, memory_bank = self.model.encoder(src, src_lengths)
        dec_states = \
            self.model.decoder.init_decoder_state(src, memory_bank, enc_states)

        #  (2) if a target is specified, compute the 'goldScore'
        #  (i.e. log likelihood) of the target under the model
        tt = torch.cuda if self.cuda else torch
        gold_scores = tt.FloatTensor(batch.batch_size).fill_(0)
        dec_out, _, _ = self.model.decoder(
            tgt_in, memory_bank, dec_states, memory_lengths=src_lengths)

        tgt_pad = self.fields["tgt"].vocab.stoi[onmt.io.PAD_WORD]
        for dec, tgt in zip(dec_out, batch.tgt[1:].data):
            # Log prob of each word.
            out = self.model.generator.forward(dec)
            tgt = tgt.unsqueeze(1)
            scores = out.data.gather(1, tgt)
            scores.masked_fill_(tgt.eq(tgt_pad), 0)
            print("scores size:",str(scores.reshape([1,-1]).squeeze().size()))
            print("gold_scores:",str(gold_scores.size()))
            gold_scores += scores.reshape([1,-1]).squeeze()
        
        return gold_scores

In [ ]:
gold_scores = torch.rand(3,3)
gold_scores.reshape([1,-1]).squeeze().unsqueeze(0)

In [ ]:
gold_scores = torch.rand(3,3)
gold_scores.reshape([1,-1]).squeeze().unsqueeze(1)

In [ ]:
gold_scores = torch.rand(3,3,3)
gold_scores.reshape([1,-1]).squeeze().unsqueeze(0)

In [ ]:
gold_scores = torch.rand(3,3,3)
gold_scores.reshape([1,-1]).squeeze().unsqueeze(0)

In [ ]:
gold_scores = torch.rand(3,3,3)
gold_scores.reshape([1,-1]).squeeze().unsqueeze(2)

# get_hyp test in Beam.py

In [ ]:
def get_hyp(self,timestep, k):
    """
    Walk back to construct the full hypothesis.
    """
    hyp, attn = [], []
    for j in range(len(self.prev_ks[:timestep]) - 1, -1, -1):
        # print("type(self.next_ys[j+1][k])",str(type(self.next_ys[j+1][k])))
        # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].dtype))
        # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].type(torch.long).dtype))
        print(j,int(k))
        print("self.next_ys[j+1][k]",str(self.next_ys[j+1][int(k)]))
        print("type self.next_ys[j+1][k]",str(self.next_ys[j+1][int(k)].type()))
        hyp.append(self.next_ys[j+1][int(k)])
        attn.append(self.attn[j][int(k)])
        k = self.prev_ks[j][int(k)]
    return hyp[::-1], torch.stack(attn[::-1])
for b in beam:
            n_best = translator.n_best
            scores, ks = b.sort_finished(minimum=n_best)
            hyps, attn = [], []
            for i, (times, k) in enumerate(ks[:n_best]):
                # print(times,k)

                hyp, att = get_hyp(b,times, k)
                hyps.append(hyp)
                attn.append(att)
            ret["predictions"].append(hyps)
            ret["scores"].append(scores)
            ret["attention"].append(attn)



In [ ]:
def get_hyp(self,timestep, k):
    """
    Walk back to construct the full hypothesis.
    """
    k=int(k)
    hyp, attn = [], []
    for j in range(len(self.prev_ks[:timestep]) - 1, -1, -1):
        # print("type(self.next_ys[j+1][k])",str(type(self.next_ys[j+1][k])))
        # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].dtype))
        # print("self.next_ys[j+1][k].dtype",str(self.next_ys[j+1][k].type(torch.long).dtype))
        print(j,k)
        print("self.next_ys[j+1][k]",str(self.next_ys[j+1][k]))
        print("type self.next_ys[j+1][k]",str(self.next_ys[j+1][k].type()))
        hyp.append(self.next_ys[j+1][k])
        attn.append(self.attn[j][k])
        k = self.prev_ks[j][k]
    return hyp[::-1], torch.stack(attn[::-1])
for b in beam:
            n_best = translator.n_best
            scores, ks = b.sort_finished(minimum=n_best)
            hyps, attn = [], []
            for i, (times, k) in enumerate(ks[:n_best]):
                # print(times,k)

                hyp, att = get_hyp(b,times, int(k))
                hyps.append(hyp)
                attn.append(att)
            ret["predictions"].append(hyps)
            ret["scores"].append(scores)
            ret["attention"].append(attn)



# _run_beam TEST in Translator.py

In [ ]:
for i, batch in enumerate(data_iter):
    print(str(i), str(batch.batch_size))
    beam_size = translator.beam_size
    batch_size = batch.batch_size
    data_type = data.data_type
    vocab = translator.fields["tgt"].vocab
    beam = []
    for __ in range(batch.batch_size):
        beam.append(onmt.translate.Beam(beam_size, n_best=translator.n_best,
                                    cuda=translator.cuda,
                                    global_scorer=translator.global_scorer,
                                    pad=vocab.stoi[onmt.io.PAD_WORD],
                                    eos=vocab.stoi[onmt.io.EOS_WORD],
                                    bos=vocab.stoi[onmt.io.BOS_WORD],
                                    min_length=translator.min_length,
                                    stepwise_penalty=translator.stepwise_penalty))

In [ ]:
data_type = data.data_type
if data_type == 'text':
    _, src_lengths = batch.src
elif data_type == 'gcn':
    _, src_lengths = batch.src
    # report_stats.n_src_words += src_lengths.sum()
    adj_arc_in, adj_arc_out, adj_lab_in, adj_lab_out, \
    mask_in, mask_out, mask_loop, mask_sent = onmt.io.get_adj(batch)
    if hasattr(batch, 'morph'):
        morph, mask_morph = onmt.io.get_morph(batch)  # [b,t, max_morph]
else:
    src_lengths = None
src = onmt.io.make_features(batch, 'src', data_type)
tgt_in = onmt.io.make_features(batch, 'tgt')[:-1]



#  (1) run the encoder on the src
if data_type == 'gcn':
    # F-prop through the model.
    if hasattr(batch, 'morph'):
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                                adj_arc_in, adj_arc_out, adj_lab_in,
                                adj_lab_out, mask_in, mask_out,
                                mask_loop, mask_sent, morph, mask_morph)
    else:
        enc_states, memory_bank = \
            translator.model.encoder(src, src_lengths,
                        adj_arc_in, adj_arc_out, adj_lab_in,
                        adj_lab_out, mask_in, mask_out,
                        mask_loop, mask_sent)

else:
    enc_states, memory_bank = translator.model.encoder(src, src_lengths)
dec_states = \
    translator.model.decoder.init_decoder_state(src, memory_bank, enc_states)

#  (2) if a target is specified, compute the 'goldScore'
#  (i.e. log likelihood) of the target under the model
tt = torch.cuda if translator.cuda else torch
gold_scores = tt.FloatTensor(batch.batch_size).fill_(0)
dec_out, _, _ = translator.model.decoder(
    tgt_in, memory_bank, dec_states, memory_lengths=src_lengths)

tgt_pad = translator.fields["tgt"].vocab.stoi[onmt.io.PAD_WORD]
for dec, tgt in zip(dec_out, batch.tgt[1:].data):
    # Log prob of each word.
    out = translator.model.generator.forward(dec)
    tgt = tgt.unsqueeze(1)
    scores = out.data.gather(1, tgt)
    print("scores",str(scores))
    scores.masked_fill_(tgt.eq(tgt_pad), 0)
    # print("gold_scores",str(gold_scores))
    # print("scores",str(scores))
    # print("size gold_scores",str(gold_scores.size()))
    # print("size scores",str(scores.size()))
    gold_scores += scores.squeeze(1)
    print(len(gold_scores))

# _from_beam TEST in Translator.py

In [ ]:
def _from_beam(self, beam):
    ret = {"predictions": [],
            "scores": [],
            "attention": []}
    for b in beam:
        n_best = self.n_best
        scores, ks = b.sort_finished(minimum=n_best)
        hyps, attn = [], []
        for i, (times, k) in enumerate(ks[:n_best]):
            hyp, att = b.get_hyp(times, k)
            hyps.append(hyp)
            attn.append(att)
        ret["predictions"].append(hyps)
        print("ret[predictions]",str(ret["predictions"]))
        print("size ret[predictions]",str(len(ret["predictions"])))
        ret["scores"].append(scores)
        ret["attention"].append(attn)
    return ret

In [ ]:
ret = _from_beam(translator,beam)

In [ ]:
ret = {"predictions": [],
        "scores": [],
        "attention": []}
for b in beam:
    n_best = translator.n_best
    scores, ks = b.sort_finished(minimum=n_best)
    hyps, attn = [], []
    for i, (times, k) in enumerate(ks[:n_best]):
        hyp, att = b.get_hyp(times, k)
        hyps.append(hyp)
        attn.append(att)
    ret["predictions"].append(hyps)
    ret["scores"].append(scores)
    ret["attention"].append(attn)

# 保留

In [ ]:
scores.squeeze(0).size()

In [ ]:
b.get_current_state()

In [ ]:
next_ys = torch.tensor([]).new_full((5,),1,dtype=torch.long)

In [ ]:
hyp = []
hyp.append(next_ys[0][0])

In [ ]:
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]
opt.cuda = opt.gpu > -1
if opt.cuda:
    # torch.cuda.set_device(opt.gpu)
    torch.cuda.set_device('cuda:0')

# Load the model.
fields, model, model_opt = \
    onmt.ModelConstructor.load_test_model(opt, dummy_opt.__dict__)

# File to write sentences to.
out_file = codecs.open(opt.output, 'w', 'utf-8')

# Test data
if opt.data_type == 'gcn':
    data = onmt.io.build_dataset_gcn(fields, opt.data_type,
                                opt.src, opt.tgt, opt.src_label,
                                opt.src_node1, opt.src_node2,
                                opt.src_morph,
                                src_dir=opt.src_dir,
                                sample_rate=opt.sample_rate,
                                window_size=opt.window_size,
                                window_stride=opt.window_stride,
                                window=opt.window,
                                use_filter_pred=False)
else:
    data = onmt.io.build_dataset(fields, opt.data_type,
                                opt.src, opt.tgt,
                                src_dir=opt.src_dir,
                                sample_rate=opt.sample_rate,
                                window_size=opt.window_size,
                                window_stride=opt.window_stride,
                                window=opt.window,
                                use_filter_pred=False)

# Sort batch by decreasing lengths of sentence required by pytorch.
# sort=False means "Use dataset's sortkey instead of iterator's".
data_iter = onmt.io.OrderedIterator(
    dataset=data, device=opt.gpu,
    batch_size=opt.batch_size, train=False, sort=False,
    sort_within_batch=True, shuffle=False)

# Translator
scorer = onmt.translate.GNMTGlobalScorer(opt.alpha,
                                            opt.beta,
                                            opt.coverage_penalty,
                                            opt.length_penalty)
translator = onmt.translate.Translator(
    model, fields,
    beam_size=opt.beam_size,
    n_best=opt.n_best,
    global_scorer=scorer,
    max_length=opt.max_length,
    copy_attn=model_opt.copy_attn,
    cuda=opt.cuda,
    beam_trace=opt.dump_beam != "",
    min_length=opt.min_length,
    stepwise_penalty=opt.stepwise_penalty)
builder = onmt.translate.TranslationBuilder(
    data, translator.fields,
    opt.n_best, opt.replace_unk, opt.tgt)

# Statistics
counter = count(1)
pred_score_total, pred_words_total = 0, 0
gold_score_total, gold_words_total = 0, 0
for i, batch in enumerate(data_iter):
    print(i)
    print(batch.src)
    print(batch.node1)
    if i==0: break